### Тетрадка для дизайна и анализа, использующая функции с модуля *well_model*

In [ ]:
import pandas as pd

from plotly import graph_objects as go
from plotly.offline import plot

import matplotlib
import matplotlib.pyplot as plt

import numpy as np
import tqdm

import seaborn as sns

from numba import jit

from scipy import interpolate

import shapely
from shapely.geometry import LineString, Point
import os

os.chdir(r"C:\Git\probability_calculations")

plt.rcParams["figure.figsize"] = (15,10)

import json

import seaborn as sns
import glob, os

VBA_VERSION = '7.28'
import well_model
import mult

In [ ]:
font = {#'family' : 'normal',
        #'weight' : 'bold',
        'size'   : 25}

matplotlib.rc('font', **font)

In [ ]:
import UniflocVBA.v7_25.python_api as python_api_7_25
api = python_api_7_25.API("UniflocVBA/v7_25/UniflocVBA_7.xlam")

import UniflocVBA.v7_28.python_api as python_api_7_28
api_new = python_api_7_28.API("UniflocVBA/v7_28/UniflocVBA_7_28.xlam")
api_new.encode_PVT()

# TODO

* Физика
    * Отдебажить VBA - избавиться от пропусков в распределении
    * Сделать качественные графики узлового анализа и прочих расчетов
    * Запуск одной скважины скорректированный
    
* Логика расчетов
    * Модульность
    * Последовательность
    * Сохранение всех результатов расчета через json
    
* Ускорение расчетов - потом
    * Multiprocessing
    * json настроек
    * модульность
    
* Автоматический подбор
    * Парсинг базы БД для выбора ЭЦН для данного распределения дебита - несколько насосов + 
    * Выбор насосов + 
    * Автоматический подбор нескольких скважин
    * Поиск начального приближения для напора
    
    
* Меньше счета, больше анализа - поймать полезность / робастность / влияние на решение
    * Когда меняется решения
    * Сравнить входные распределения параметров
    * Сделать разные входные распределения
    
    
* Итоговый вывод
    * Полный pipeline по одной кнопке - исходные данные - подобрать - решение.
    * Таблица сводная с EVM по к насосам и м напорам
    
    
    
Лог    
13.05 - дебаг моделей - настройка навого pipeline

14.05 - первый расчет по новому пайплайну - определение слабых мест и мест для развития - обсуждения с РА

15.05 - дебаг физики и качественная визуализация 

16.05 - первая версия пайплайна и подбора

17.05 - анализ данных - запуск дополнительного расчета

18.05 - анализ данных - визуализация
    
19.05 - пропуск

20.05 - переход на UniflocVBA_7_27, дебаг физики

21.05 - переход на UniflocVBA_7_28
    


# start: boost

In [ ]:
p_ar = np.arange(1, 300, 1)
t_ar = np.arange(1, 200, 5)

from itertools import product


PVT_prop = api_new.encode_PVT()
res = []
for p, t in tqdm.tqdm(product(p_ar, t_ar), total = int(len(p_ar)*len(t_ar))):
    this_dict = json.loads(api_new.PVT_calc(p, t,   PVT_prop))
    this_dict['p'] = p
    this_dict['t'] = t
    res.append(pd.DataFrame(this_dict, index = [0]))

    

In [ ]:
res_df = pd.concat(res, ignore_index = True)

In [ ]:
res_df

In [ ]:
def norm(x, train_stats):
    return (x - train_stats['mean']) / train_stats['std']
def predict_parameter_via_nn(x_train, x_test, y_train, y_test=None, use_in_loop=True):
    model = None
    EPOCHS = 1000

    train_dataset = x_train.copy()
    test_dataset = x_test.copy()

    train_dataset = train_dataset.reset_index(drop=True)

    test_dataset = test_dataset.reset_index(drop=True)

    train_labels = y_train.copy()

    # print(f"train_dataset = {x_train.shape}")
    # print(f"test_dataset = {x_test.shape}")
    # print(f"train_labels = {y_train.shape}")
    # print(f"test_labels = {y_test.shape}")

    train_stats = train_dataset.describe()
    # train_stats.pop("MPG")
    train_stats = train_stats.transpose()

    if type(y_train) == pd.DataFrame:
        target_name = y_train.columns[0]
        train_labels = train_labels.pop(target_name)
    else:
        target_name = y_train.name
        train_labels = train_labels

    normed_train_data = norm(train_dataset, train_stats)
    normed_test_data = norm(test_dataset, train_stats)

    # print(normed_train_data.describe())

    model = build_model(len(train_dataset.keys()))

    # print(model.summary())

    # print(f"normed_train_data = {normed_train_data.shape}")

    # print(f"train_labels = {train_labels.shape}")

    # The patience parameter is the amount of epochs to check for improvement
    early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=20)

    early_history = model.fit(normed_train_data, train_labels,
                              epochs=EPOCHS, validation_split=0.2, verbose=1,
                              callbacks=[
                                  early_stop]
                              )

    test_predictions = model.predict(normed_test_data).flatten()

    if not use_in_loop:
        test_labels = y_test.copy()
        test_labels = test_labels.pop(target_name)

        plotter = tfdocs.plots.HistoryPlotter(smoothing_std=2)

        plotter.plot({'Early Stopping': early_history}, metric="mae")
        # plt.ylim([0, 10])
        plt.ylabel(f"MAE [{target_name}]")
        plt.show()

        loss, mae, mse = model.evaluate(normed_test_data, test_labels, verbose=2)

        print("Testing set Mean Abs Error: {:5.2f} MPG".format(mae))

        a = plt.axes(aspect='equal')
        plt.scatter(test_labels, test_predictions)
        plt.xlabel(f"True Values [{target_name}]")
        plt.ylabel(f"Predictions [{target_name}]")
        lims = [0, max(test_predictions)]
        plt.xlim(lims)
        plt.ylim(lims)
        _ = plt.plot(lims, lims)
        plt.show()

        error = test_predictions - test_labels
        plt.hist(error, bins=25)
        plt.xlabel(f"Prediction Error [{target_name}]")
        _ = plt.ylabel("Count")
        plt.show()
    # return test_labels, test_predictions
    y_test = test_predictions
    return y_test, model, 'neural network', early_history
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

def build_model(input_shape):
    model = keras.Sequential([
        layers.Dense(10, activation='relu', input_shape=[input_shape]),
        layers.Dense(10, activation='relu'),
        layers.Dense(1)
    ])

    optimizer = tf.keras.optimizers.RMSprop(0.001)

    model.compile(loss='mse',
                  optimizer=optimizer,
                  metrics=['mae', 'mse'])
    return model

In [ ]:
res_df_train = res_df.sample(frac=0.8)
res_df_test = res_df[~res_df.index.isin(res_df_train.index)]
res_df_train

In [ ]:
res_df_test

In [ ]:
x_train = res_df_train[['p', 't']]
x_test = res_df_test[['p', 't']]

y_train = res_df_train[['rs_m3m3']]
y_test = res_df_test[['rs_m3m3']]


In [ ]:
prediction, fitted_model, model_name, early_history = predict_parameter_via_nn(x_train, x_test, y_train,
                                                               
                                                               None, 
                                                               use_in_loop=True)

In [ ]:
def plot_loss(history):
    plt.plot(history.history['loss'], label='loss')
    plt.plot(history.history['val_loss'], label='val_loss')
    plt.ylim([0, 10])
    plt.xlabel('Epoch')
    plt.ylabel('Error [MPG]')
    plt.legend()
    plt.grid(True)

In [ ]:
plot_loss(early_history)



In [ ]:
np.mean((abs(prediction) - y_test.values.flatten()) / y_test.values.flatten())*100

In [ ]:
from sklearn import metrics

In [ ]:
plt.plot(abs(prediction), y_test.values.flatten(), 'o')
plt.title(metrics.r2_score(y_test.values.flatten(), abs(prediction)))

In [ ]:
fitted_model.predict(res_df_test[['p', 't']]).flatten()

In [ ]:
prediction

In [ ]:
x = norm(res_df_test[['p', 't']], res_df_test[['p', 't']].describe().T)
y_pred, y = fitted_model.predict(x).flatten(), res_df_test['rs_m3m3']

plt.plot(y,  y_pred, 'o')
plt.title(metrics.r2_score(y, abs(y_pred)))


In [ ]:
x = norm(res_df[['p', 't']], res_df[['p', 't']].describe().T)


In [ ]:
%%timeit

y_pred = fitted_model.predict(x)

In [ ]:
y_pred

In [ ]:
%%timeit
for i in tqdm.tqdm(x.index):
    this_x = res_df[res_df.index == i]
    y_pred = api_new.PVT_calc(this_x['p'].values[0], this_x['t'].values[0], PVT_prop)[0][0]

In [ ]:
def predict_parameter_via_lightgbm(x_train, x_test, y_train):
    x_train = x_train.values
    x_test = x_test.values
    y_train = y_train.values

    #sc = StandardScaler()
    #sc.fit(x_train)
    #x_train = sc.transform(x_train)
    #x_test = sc.transform(x_test)

    clf = LGBMRegressor(
        # n_estimators=50,
        # n_estimators=i,
        # num_boost_round=i,
        # learning_rate=1/i,
        # num_leaves=i,
        # colsample_bytree=1 / 2.5, #TODO спросить, для нее важен порядок колонок dp на клапане в начале или в середине
        # learning_rate=0.01,
        # subsample=1/i,
        # num_leaves=i,
        # max_depth=i**2+1,
        # reg_alpha=1/i,
        # reg_lambda=1/i,
        # min_split_gain=1/i,
        # min_child_weight=i,
    )


    clf.fit(
        x_train, y_train
    )

    y_pred = clf.predict(x_test)
    return y_pred, clf, 'lightgbm'

from lightgbm import LGBMRegressor


In [ ]:
x_train = res_df_train[['p', 't']]
x_test = res_df_test[['p', 't']]

y_train = res_df_train[['rs_m3m3']]
y_test = res_df_test[['rs_m3m3']]


In [ ]:
prediction, fitted_model, model_name = predict_parameter_via_lightgbm(x_train, x_test, y_train)

In [ ]:
plt.plot(abs(prediction), y_test.values.flatten(), 'o')
plt.title(metrics.r2_score(y_test.values.flatten(), abs(prediction)))

In [ ]:
%%timeit

y_pred = fitted_model.predict(res_df[['p', 't']])

In [ ]:
fitted_model

In [ ]:
y_pred, y = fitted_model.predict(res_df[['p', 't']]), res_df['rs_m3m3'].values.flatten()
plt.plot(abs(y_pred), y, 'o')
plt.title(metrics.r2_score(y, abs(y_pred)))

In [ ]:
len(y_pred)

In [ ]:
import m2cgen as m2c


In [ ]:
code = m2c.export_to_python(fitted_model)
code

In [ ]:
f = open("ml_model.py", "w")
f.write(code)
f.close()

In [ ]:
import ml_model

In [ ]:
x = norm(res_df[['p', 't']], res_df[['p', 't']].describe().T)


In [ ]:
%%timeit
for i in tqdm.tqdm(x.index):
    this_x = res_df[res_df.index == i]
    y_pred = ml_model.score([this_x['p'].values[0], this_x['t'].values[0]])

In [ ]:
ml_model.score([20, 20])

# end. boost

# Функции для работы с НРХ

In [ ]:
plot_pump_curve = well_model.plot_pump_curve

interp_df = well_model.interp_df

calc_num_stages = well_model.calc_num_stages


# Функция для расчета IPR по Вогелю (быстрая) Qliq

In [ ]:
calc_QliqVogel_m3Day = well_model.calc_QliqVogel_m3Day

# Создание нормального распределения

In [ ]:
create_normal_dist = well_model.create_normal_dist

dist = create_normal_dist(2, 1, 10**6, plot=False)

# Расчетные функции

## Базовые параметры расчета

In [ ]:
params = mult.params.copy()


## Расчет физической модели

### Определения функции calc_esp

In [ ]:
debug = True

calc_esp_old = well_model.calc_esp_old

calc_esp_new = well_model.calc_esp_new


#### ЭЦН UniflocVBA 7.25 тест

In [ ]:
p_esp_dis, gas_fraction_intake, eff, head_esp, power_esp, r, q_mix_pump_mean = calc_esp_new(params, api.PVT_encode_string(),
                                                                                            50, 110, m_api=api)
r

#### ЭЦН UniflocVBA 7.28 тест

In [ ]:
calc_esp_new_7_28 = well_model.calc_esp_new_7_28

In [ ]:
p_esp_dis, gas_fraction_intake, eff, head_esp, power_esp, r, q_mix_pump_mean = calc_esp_new_7_28(params, api.PVT_encode_string(), 
                                                                                                 50, 110, m_api=api_new)
r    

### Определение функции для построения НРХ в условиях работы

In [ ]:
plot_pump_curves = well_model.plot_pump_curves

#### Сравнение ЭЦН для 7.25 и 7.28

In [ ]:
p_intake, t_intake = 50, 70
params['ksep_fr'] = 0.7
params['qliq_sm3day_range'] = np.arange(1, 80, 5)
d = params.copy()
str_PVT_tube = api.PVT_encode_string(gamma_gas=d['gamma_gas'],
                gamma_oil=d['gamma_oil'],
                gamma_wat=d['gamma_wat'],
                rsb_m3m3=d['rsb_m3m3'],
                rp_m3m3=d['rp_m3m3'],
                pb_atma=d['pb_atma'],
                t_res_C=d['t_res_C'],
                bob_m3m3=d['bob_m3m3'],
                muob_cP=d['muob_cP'],
                PVTcorr=d['PVTcorr'],
                ksep_fr=d['ksep_fr'],
                p_ksep_atma=float(p_intake),
                t_ksep_C=float(t_intake),
                gas_only=False)
print(f"UniflocVBA 7.28")
plot_pump_curves(params, None, None,  None,  p_intake, t_intake, None, q_mix_pump_mean, 
                 qliq_on_surface=True, vba_version='7.28', api=api, api_new=api_new)
print(f"UniflocVBA 7.25")
plot_pump_curves(params, None, None,  None, p_intake, t_intake, str_PVT_tube, q_mix_pump_mean, 
                 qliq_on_surface=True, vba_version='7.25', api=api, api_new=api_new)

### Определение функции для построения КРД и КРТ

In [ ]:
plot_well_curves = well_model.plot_well_curves

### Определения функции calc_model

#### Модель UniflocVBA 7.25

In [ ]:
calc_model = well_model.calc_model

In [ ]:
p_dis, p_esp_dis, str_PVT_tube, p_intake, t_intake,  eff, head_esp, power_esp, gas_fraction_intake,\
                    casing_pipe, tube_pipe, q_mix_pump_mean, esp_df= calc_model(params, m_api=api)

In [ ]:
pd.DataFrame(casing_pipe)

In [ ]:
plot_well_curves(p_dis, p_esp_dis, str_PVT_tube, p_intake, t_intake,  eff, head_esp, 
                 power_esp, gas_fraction_intake,
                    casing_pipe, tube_pipe, params)

#### Модель UniflocVBA 7.28

In [ ]:
calc_model_new_7_28 = well_model.calc_model_new_7_28

In [ ]:
params['temp_method'] = 2

In [ ]:
p_dis, p_esp_dis, str_PVT_tube, p_intake, t_intake,  eff, head_esp, power_esp, gas_fraction_intake,\
                    casing_pipe, tube_pipe, q_mix_pump_mean, esp_df= calc_model_new_7_28(params, m_api=api_new)

In [ ]:
plot_well_curves(p_dis, p_esp_dis, str_PVT_tube, p_intake, t_intake,  eff, head_esp, power_esp, gas_fraction_intake,
                    casing_pipe, tube_pipe, params)

### Построение VLP

In [ ]:
test_params = params.copy()
lr = []


q_range = list(range(1, 150,2))
for i in q_range:
    test_params['qliq_sm3day'] = i  

    r = calc_model_new_7_28(test_params, m_api=api_new)
    #r = calc_model(test_params, m_api=api)
    
    lr.append(r)
    

In [ ]:
plt.plot(q_range, [x[0] for x in lr], label = 'pdis')
plt.plot(q_range, [x[1] for x in lr], label = 'p_esp_dis')
plt.plot(q_range, [x[3] for x in lr], label = 'p_intake')

plt.legend()

In [ ]:
plt.plot(q_range, [x[-2] for x in lr], label = 'q_mix_pump_mean')

plt.legend()

In [ ]:
plt.plot(q_range, [x[-5] for x in lr], label = 'gas_fraction_intake')

plt.legend()

In [ ]:
plt.plot(q_range, [x[0] for x in lr], label = 'pdis')
plt.plot(q_range, [x[1] for x in lr], label = 'p_esp_dis')
plt.plot(q_range, [x[3] for x in lr], label = 'p_intake')

plt.legend()

### Функции для построения НРХ в условиях работы

In [ ]:
params['qliq_sm3day_range'] = range(1, 100)
plot_pump_curves(params, head_esp, power_esp, eff, p_intake, t_intake, str_PVT_tube, q_mix_pump_mean, qliq_on_surface=False,
                api=api, api_new=api_new)
plot_pump_curves(params, head_esp, power_esp, eff, p_intake, t_intake, str_PVT_tube, q_mix_pump_mean, qliq_on_surface=True,
                api=api, api_new=api_new)

## Определение основной расчетной функции узлового анализа

In [ ]:
calc_all = well_model.calc_all

save_in_df = well_model.save_in_df



In [ ]:
#os.system("taskkill /f /im EXCEL.EXE")

# Проверка рабочей функции

In [ ]:
params_saved = params.copy()
params_saved

In [ ]:
matplotlib.rcParams.update({'font.size': 12})
params['n_dots_for_nodal']  = 20
params['calc_esp_new'] = 1
params['hydr_corr'] = 1
params = mult.params.copy()
params['freq_Hz'] = 60
#params['ESP_gas_correct'] = 5
#params['pi_sm3dayatm'] = 0.9
#params['h_pump_m'] = 2661
#params['ksep_fr'] = 0.8
#params['p_wh_atm'] = 15
params['num_stages'] = 604


casing_pipe, tube_pipe, h_mes, head_esp, eff, power_esp, q_liq, status, gas_fraction_intake,\
p_bhp_atm, p_dis, p_esp_dis, json_params, q_mix_pump_mean, esp_df  = calc_all(params, debug=1, vba_version='7.28',
                                                                             api=api, api_new=api_new)

In [ ]:
esp_df

In [ ]:
params_saved = params.copy()
params_saved

In [ ]:
matplotlib.rcParams.update({'font.size': 12})
params['n_dots_for_nodal']  = 20
params['calc_esp_new'] = 1
params['hydr_corr'] = 1

casing_pipe, tube_pipe, h_mes, head_esp, eff, power_esp, q_liq, status, gas_fraction_intake,\
p_bhp_atm, p_dis, p_esp_dis, json_params, q_mix_pump_mean, esp_df  = calc_all(params, debug=1, vba_version='7.25',
                                                                             api=api, api_new=api_new)
casing_pipe, tube_pipe, h_mes, head_esp, eff, power_esp, q_liq, status, gas_fraction_intake,\
p_bhp_atm, p_dis, p_esp_dis, json_params, q_mix_pump_mean, esp_df  = calc_all(params, debug=1, vba_version='7.28',
                                                                             api=api, api_new=api_new)

In [ ]:
matplotlib.rcParams.update({'font.size': 12})
params['n_dots_for_nodal']  = 20
params['calc_esp_new'] = 1
params['hydr_corr'] = 1
casing_pipe, tube_pipe, h_mes, head_esp, eff, power_esp, q_liq, status, gas_fraction_intake,\
p_bhp_atm, p_dis, p_esp_dis, json_params, q_mix_pump_mean, esp_df  = calc_all(params, debug=1, vba_version='7.25',
                                                                             api=api, api_new=api_new)
casing_pipe, tube_pipe, h_mes, head_esp, eff, power_esp, q_liq, status, gas_fraction_intake,\
p_bhp_atm, p_dis, p_esp_dis, json_params, q_mix_pump_mean, esp_df  = calc_all(params, debug=1, vba_version='7.28',
                                                                             api=api, api_new=api_new)

In [ ]:
params['n_dots_for_nodal']  = 20
params['calc_esp_new'] = 1
params['esp_head_m'] = 1800
params['num_stages'] = calc_num_stages(params, api=api, api_new=api_new, vba_version=VBA_VERSION)
casing_pipe, tube_pipe, h_mes, head_esp, eff, power_esp, q_liq, status, gas_fraction_intake,\
p_bhp_atm, p_dis, p_esp_dis, json_params, q_mix_pump_mean, esp_df  = calc_all(params, debug=1, vba_version='7.28',
                                                                             api=api, api_new=api_new)

In [ ]:
#params['calc_esp_new'] = 0
#casing_pipe, tube_pipe, h_mes, head_esp, eff, power_esp, q_liq, status, gas_fraction_intake,\
#p_bhp_atm, p_dis, p_esp_dis = calc_all(params)

## Анализ системы (анализ чувствительности для узлового анализа)

In [ ]:
sens_params = params.copy()
sens_params

### Коэффициент продуктивности

In [ ]:
res_list2 = []
sens_params['n_dots_for_nodal']  = 20
sens_params['pump_id'] = 2753 #100 проблемная
sens_params['esp_head_m'] = 1500
sensed_param = np.arange(0.5, 3, 0.3)
stack_values = []
for i in tqdm.tqdm(sensed_param):
    print('\n')
    print('i', i)
    sens_params['pi_sm3dayatm'] = i
    res = calc_all(sens_params, debug=1, vba_version='7.25', api=api, api_new=api_new)
    casing_pipe, tube_pipe, h_mes, head_esp, eff, power_esp, q_liq, status, gas_fraction_intake,\
    p_bhp_atm, p_dis, p_esp_dis, json_params, q_mix_pump_mean, esp_df = res
    res_list2.append(res)
    this_values = ('head_esp',  head_esp, 'eff', eff, 'power_esp', power_esp, 'q_liq', q_liq,  'gas_fraction_intake', gas_fraction_intake,
        'p_bhp_atm', p_bhp_atm, 'p_dis', p_dis, 'p_esp_dis',  p_esp_dis, 'q_mix_pump_mean', q_mix_pump_mean)
    
    stack_values.append(this_values)
    print(this_values)

In [ ]:
fig = plt.Figure()
plt.plot([x[6] for x in res_list2],
         [x[4] for x in res_list2], 'o-', label = 'КПД, д.ед.')
plt.xlabel('Дебит жидкости в поверхностных условиях, м3/сут')
plt.ylabel('КПД, д.ед.')
plt.title('Системный анализ чувствительности')
plt.show()


for i in [
    [ [x[6] for x in res_list2], 'Дебит жидкости в поверхностных условиях, м3/сут'],
    [sensed_param, 'Кпрод, м3/сут/атм'],
    [  [x[13] for x in res_list2], 'Средний расход ГЖС через ЭЦН, м3/сут']
    ]:
    sensed_x = i[0]
    sensed_x_name = i[1]
    
    
    fig = plt.Figure()
    plt.plot(sensed_x,
             [x[11] for x in res_list2], 'o-', label = 'Давление на выкиде ЭЦН по ЭЦН, атм')
    plt.plot(sensed_x,
             [x[10] for x in res_list2], 'o-', label = 'Давление на выкиде ЭЦН по трубе, атм')
    plt.plot(sensed_x,
             [x[9] for x in res_list2], 'o-', label = 'Забойное давление, атм')
    plt.plot(sensed_x,
             [x[14][3][3] for x in res_list2], 'o-', label = 'Давление на приеме ЭЦН, атм')
    
    plt.xlabel(sensed_x_name)
    plt.ylabel('Давление, атм')
    plt.title('Системный анализ чувствительности')
    plt.legend()
    plt.show()


plt.plot(sensed_param,
         [x[4] for x in res_list2], 'o-', label = 'КПД, д.ед.')
plt.xlabel('Кпрод, м3/сут/атм')
plt.ylabel('КПД')
plt.title('Системный анализ чувствительности')
plt.legend()
plt.show()



plt.plot( [x[4] for x in res_list2],
         [x[8] for x in res_list2], 'o-', label = 'Доля газа на приеме ЭЦН (после сепарации), д.ед.')
plt.xlabel('Дебит жидкости в поверхностных условиях, м3/сут')
plt.ylabel('Доля газа на приеме ЭЦН (после сепарации), д.ед.')
plt.title('Системный анализ чувствительности')
plt.legend()
plt.show()

In [ ]:
sensed_param

In [ ]:
sens_params

In [ ]:
i_iter = -1
sens_params['bob_m3m3'] = 1.2
sens_params['muob_cP'] = 1
sens_params['diam_list_mm_tube'] = 60

sens_params['n_dots_for_nodal']  = 20
sens_params['pump_id'] = 2753 #100 проблемная
sens_params['esp_head_m'] = 1500
sens_params['pi_sm3dayatm'] = sensed_param[i_iter]
sens_params['hydr_corr'] = 1
sens_params['ksep_fr'] = 0
res = calc_all(sens_params, debug=1, vba_version='7.28', api=api, api_new=api_new)
casing_pipe, tube_pipe, h_mes, head_esp, eff, power_esp, q_liq, status, gas_fraction_intake,\
p_bhp_atm, p_dis, p_esp_dis, json_params, q_mix_pump_mean, esp_df = res

print('from loop')
print(stack_values[i_iter])
print('this')
print(( 'head_esp',  head_esp, 'eff', eff, 'power_esp', power_esp, 'q_liq', q_liq,  'gas_fraction_intake', gas_fraction_intake,
'p_bhp_atm', p_bhp_atm, 'p_dis', p_dis, 'p_esp_dis',  p_esp_dis, 'q_mix_pump_mean', q_mix_pump_mean))



### Насосы

In [ ]:
sens_params = params.copy()
sens_params['qliq_sm3day_range'] = np.arange(1, 200, 5)
for i in [2753, #страння характеристика эцн-100 - из-за округлений
          #1460, #125
           # 936, #83
          #1153, #80
         ]:

    sens_params['pump_id'] = i
    plot_pump_curves(sens_params, None, None, None, 50, 80, api.PVT_encode_string(ksep_fr=0.7, 
                                                                                  p_ksep_atma=50,
                                                                                 t_ksep_C=80), None, qliq_on_surface=True,
                                                                                    vba_version='7.28', api=api,
                                                                                    api_new=api_new)
    plot_pump_curves(sens_params, None, None, None, 50, 80, api.PVT_encode_string(ksep_fr=0.7, 
                                                                                  p_ksep_atma=50,
                                                                                 t_ksep_C=80), None, qliq_on_surface=False,
                                                                                    vba_version='7.28', api=api,
                                                                                    api_new=api_new)
    plot_pump_curves(sens_params, None, None, None, 50, 80, api.PVT_encode_string(ksep_fr=0.7, 
                                                                                  p_ksep_atma=50,
                                                                                 t_ksep_C=80), None, qliq_on_surface=True,
                                                                                    vba_version='7.25', api=api,
                                                                                    api_new=api_new)
    plot_pump_curves(sens_params, None, None, None, 50, 80, api.PVT_encode_string(ksep_fr=0.7, 
                                                                                  p_ksep_atma=50,
                                                                                 t_ksep_C=80), None, qliq_on_surface=False,
                                                                                    vba_version='7.25', api=api,
                                                                                    api_new=api_new) 
    
    

# Расчет

## Определение неопределенности работы пласта

In [ ]:
params = mult.params
params

In [ ]:
os.chdir(r"C:\Git\probability_calculations")

debug=1

create_q_dist = well_model.create_q_dist

all_stats_q_new, pi_mc, dist_p_res = create_q_dist(params, pi_mean = params['pi_sm3dayatm'], pi_std = 0.10, 
                  pres_std = 10,
                  num_simulations= 1_00_000, debug=debug)

#### Сохранение распределений

In [ ]:
distr_json = {'all_stats_q_new': list(all_stats_q_new), 
             'pi_mc': list(pi_mc),
              'dist_p_res': list(dist_p_res)
             
             
             }

In [ ]:
with open('distr.txt', 'w') as json_file:
    json.dump(distr_json, json_file)

#### Загрузка распределений


In [ ]:
with open('distr_wide.txt') as json_file:
    data = json.load(json_file)
all_stats_q_new, pi_mc, dist_p_res = data['all_stats_q_new'], data['pi_mc'], data['dist_p_res']
plt.hist(all_stats_q_new,density=True,  alpha=0.9, bins = 100, label = f"1 вариант. $\sigma_1$. Qж ср. = {round(np.mean(all_stats_q_new), 1)} м3/сут")


#with open('distr_narrow.txt') as json_file:
#    data = json.load(json_file)
#all_stats_q_new, pi_mc, dist_p_res = data['all_stats_q_new'], data['pi_mc'], data['dist_p_res']
#plt.hist(all_stats_q_new,density=True, alpha=0.8, bins = 100, label = f"2 вариант. $\sigma_2 = \sigma_1/4$. Qж ср. = {round(np.mean(all_stats_q_new), 3)} м3/сут")


plt.axvline(x=np.quantile(all_stats_q_new, q=0.5), c = 'r')

plt.xlabel('Дебит жидкости, м3/сут')
plt.ylabel('Плотность вероятности')
plt.title('Распределение дебита жидкости')

plt.legend(loc='best', bbox_to_anchor=(0.85, -0.1))
plt.show()

In [ ]:
with open('distr_wide.txt') as json_file:
    data = json.load(json_file)
all_stats_q_new, pi_mc, dist_p_res = data['all_stats_q_new'], data['pi_mc'], data['dist_p_res']
plt.hist(pi_mc,density=True,  alpha=0.9, bins = 100, label = f"1 вариант. $\sigma_1$. Qж ср. = {round(np.mean(pi_mc), 1)} м3/сут/атм")


#with open('distr_narrow.txt') as json_file:
#    data = json.load(json_file)
#all_stats_q_new, pi_mc, dist_p_res = data['all_stats_q_new'], data['pi_mc'], data['dist_p_res']
#plt.hist(all_stats_q_new,density=True, alpha=0.8, bins = 100, label = f"2 вариант. $\sigma_2 = \sigma_1/4$. Qж ср. = {round(np.mean(all_stats_q_new), 3)} м3/сут")


plt.axvline(x=np.quantile(pi_mc, q=0.5), c = 'r')

plt.xlabel('Коэффициент продуктивности, м3/сут/атм')
plt.ylabel('Плотность вероятности')
plt.title('Распределения коэффициента продуктивности')

plt.legend(loc='best', bbox_to_anchor=(0.85, -0.1))
plt.show()

In [ ]:
np.std(pi_mc), np.mean(pi_mc), np.std(dist_p_res), np.mean(dist_p_res), np.std(all_stats_q_new), np.mean(all_stats_q_new)

### Сравнение распределений

In [ ]:
samples = []
for i in tqdm.tqdm(range(1000)):
    val = np.random.choice(all_stats_q_new, replace=True)
    samples.append(val)
    
samples =     np.array(samples)
print('end')

In [ ]:
samples2 = []
for i in tqdm.tqdm(range(1000)):
    val = np.random.choice(all_stats_q_new, replace=True)
    samples2.append(val)
    
samples2 =     np.array(samples2)
print('end')

In [ ]:
fig, ax = plt.subplots()

ax = pd.Series(all_stats_q_new).plot.kde(label = 'Исходное распределение. ' + f"Qж ср = {round(np.mean(all_stats_q_new), 3)} м3/сут")

#count, bins, ignored = plt.hist(all_stats_q_new, 300, [0, 400], density=True, 
#                                label=f"Qж ср = {round(np.mean(all_stats_q_new), 3)} м3/сут")

plt.axvline(x=np.quantile(all_stats_q_new, q=0.5), c = 'r')

#count, bins, ignored = plt.hist(samples, 300, [0, 400], density=True, 
#                                label=f"Qж ср = {round(np.mean(samples), 3)} м3/сут")

ax = pd.Series(samples).plot.kde(label =  f"Выборка. Qж ср = {round(np.mean(samples), 3)} м3/сут")


plt.axvline(x=np.quantile(samples, q=0.5), c = 'r')

ax = pd.Series(samples2).plot.kde(label =  f"Выборка2. Qж ср = {round(np.mean(samples2), 3)} м3/сут")


plt.axvline(x=np.quantile(samples2, q=0.5), c = 'r')


plt.title('Распределение дебита жидкости, м3/сут')
plt.xlabel('Дебит жидкости, м3/сут')
plt.ylabel('Плотность вероятности')
ax.legend()
plt.show()


#### Построение распределения 

In [ ]:
fig, ax = plt.subplots()

ax = pd.Series(all_stats_q_new).plot.kde(label = 'kde_init' + f" Qж ср = {round(np.mean(all_stats_q_new), 3)} м3/сут")

#count, bins, ignored = plt.hist(all_stats_q_new, 300, [0, 400], density=True, 
#                                label=f"Qж ср = {round(np.mean(all_stats_q_new), 3)} м3/сут")

plt.axvline(x=np.quantile(all_stats_q_new, q=0.5), c = 'r')

count, bins, ignored = plt.hist(samples, 300, [0, 400], density=True, 
                                label=f"Qж ср = {round(np.mean(samples), 3)} м3/сут")

ax = pd.Series(samples).plot.kde(label = 'kde_sample' + f"Qж ср = {round(np.mean(samples), 3)} м3/сут")


plt.axvline(x=np.quantile(samples, q=0.5), c = 'r')


plt.title('Распределение дебита жидкости, м3/сут')
plt.xlabel('Дебит жидкости, м3/сут')
plt.ylabel('Плотность вероятности')
ax.legend()
plt.show()


### Поиск подходящих насосов

In [ ]:
os.chdir(r'C:\Git\probability_calculations')
with open('ESP_json.db', 'r') as outfile:
    file = outfile
    db = json.load(outfile)

db_df = pd.DataFrame(db)
db_df = db_df.T  

number_of_point_in_curves = []
for i in db_df.index:
    this_pump = db_df.loc[i]
    number_of_point_in_curves.append(len(this_pump['rate_points']))
    
db_df['number_of_point_in_curves'] = number_of_point_in_curves



In [ ]:
q_median = np.quantile(all_stats_q_new, q=0.5)
q_std = np.std(all_stats_q_new)
q_p10 = np.quantile(all_stats_q_new, q=0.10)
q_p90 = np.quantile(all_stats_q_new, q=0.90)
q_std

In [ ]:
db_df_for_this_uncertainty = db_df[db_df['number_of_point_in_curves']>15]
db_df_for_this_uncertainty = db_df_for_this_uncertainty[db_df_for_this_uncertainty['rate_opt_min_sm3day']>q_p10-q_std]
db_df_for_this_uncertainty = db_df_for_this_uncertainty[db_df_for_this_uncertainty['rate_opt_max_sm3day']<q_p90+3*q_std]

db_df_for_this_uncertainty

In [ ]:
for i in range(db_df_for_this_uncertainty.shape[0]):
    this_pump = db_df_for_this_uncertainty.iloc[i]
    plot_pump_curve(q_arr = np.array(this_pump['rate_points']), 
        h_esp_arr = np.array(this_pump['head_points']),
        power_esp_arr = np.array(this_pump['power_points']), 
        efficiency_esp_arr = np.array(this_pump['eff_points']),
             z = 1,
                    esp_name = this_pump['name'], f=50)

In [ ]:
db_df_for_this_uncertainty['rate_nom_sm3day'].plot(kind = 'hist', bins = 100)
plt.xlabel('Номинальная подача, м3/сут')
plt.ylabel('Число ЭЦН')
plt.title('Распределение выбранных ЭЦН по подаче для текущих условий работы пласта')

In [ ]:
rates_unique = db_df_for_this_uncertainty['rate_nom_sm3day'].unique()
dq_for_group = (max(rates_unique) - min(rates_unique)) /5
rates_unique

In [ ]:
chosen_pumps = []
for i in range(5):
    q_left = min(rates_unique) + dq_for_group*i
    q_right = min(rates_unique) + dq_for_group*(i+1)
    print(q_left, q_right)
    if i == 0:
        pumps_in_group = db_df_for_this_uncertainty[(db_df_for_this_uncertainty['rate_nom_sm3day'] <= q_right) &
                                                   (db_df_for_this_uncertainty['rate_nom_sm3day'] >= q_left)]
    else:
        pumps_in_group = db_df_for_this_uncertainty[(db_df_for_this_uncertainty['rate_nom_sm3day'] <= q_right) &
                                           (db_df_for_this_uncertainty['rate_nom_sm3day'] > q_left)]
    pumps_in_group.loc[:, 'max_eff'] = pumps_in_group['eff_points'].copy().apply(lambda x: max(x))
    pumps_in_group = pumps_in_group.head(3)
    this_pumps = pumps_in_group[pumps_in_group['max_eff'] > pumps_in_group['max_eff'].max()*0.5]
    #this_pumps = pumps_in_group
    chosen_pumps.append(this_pumps)
result_pumps = pd.concat(chosen_pumps)

In [ ]:
result_pumps

In [ ]:
for i in range(result_pumps.shape[0]):
    this_pump = result_pumps.iloc[i]
    print(this_pump['rate_nom_sm3day'], this_pump['ID'])
    plot_pump_curve(q_arr = np.array(this_pump['rate_points']), 
        h_esp_arr = np.array(this_pump['head_points']),
        power_esp_arr = np.array(this_pump['power_points']), 
        efficiency_esp_arr = np.array(this_pump['eff_points']),
             z = 1,
                    esp_name = this_pump['name'], f=50)
    

## Основной цикл: Определения режима работы для выбранного(ых) насоса(ов) !!!

In [ ]:
os.chdir(r"C:\Git\probability_calculations")

params = mult.params

with open('distr.txt') as json_file:
    data = json.load(json_file)
all_stats_q_new, pi_mc, dist_p_res = data['all_stats_q_new'], data['pi_mc'], data['dist_p_res']


os.chdir(r"C:\Git\probability_calculations\calc_new")

num_simulations = 3

### для массового расчета

pumps_ids = [2089, #160,
             1460, #125
            #pump_id = 1461 #200
             2753, #100
             #1868, #80
             #2289, #60
            ]
#pumps_heads = [1000, 1800] #1 запуск

pumps_heads = [1300, 1600]

# для одного расчета
pumps_ids = [1153, #80
            2753, #100 проблемная
            
            ]
pumps_heads = [1500]

#legacy
#params['pump_id'] =  936 #80 
#params['pump_id']  = 1460 #125
#params['pump_id']  = 1461 #200
#params['pump_id']  = 2799 #60   
#params['pump_id'] = 685 # 89

#params['esp_head_m'] = 1800

params['n_dots_for_nodal'] = 15
params['calc_esp_new'] = 1


run_design = well_model.run_design  
results = run_design(params, pumps_ids, pumps_heads, pi_mc, dist_p_res, debug=1, num_simulations=1, api=api, api_new=api_new,
              vba_version='7.28')


## Анализ результатов расчета (в т.ч. массового)

In [ ]:
os.chdir(r"C:\Git\probability_calculations")

import UniflocVBA.v7_25.python_api as python_api_7_25
api = python_api_7_25.API("UniflocVBA/v7_25/UniflocVBA_7.xlam")

import UniflocVBA.v7_28.python_api as python_api_7_28
api_new = python_api_7_28.API("UniflocVBA/v7_28/UniflocVBA_7_28.xlam")
api_new.encode_PVT()

In [ ]:
with open('ESP_json.db', 'r') as outfile:
    file = outfile
    db = json.load(outfile)

db_df = pd.DataFrame(db)
db_df = db_df.T  

In [ ]:
path_to_dir = r'C:\Git\probability_calculations' + '\\'
os.chdir(path_to_dir + 'calc_new_24_7_25')
os.chdir(path_to_dir + 'calc_20')

os.chdir(path_to_dir + 'calc_new')


#os.chdir(path_to_dir + 'calc_22_7_25')

#os.chdir(path_to_dir + 'calc_25_7_28')
os.chdir(path_to_dir + 'calc_22_7_28')
os.chdir(path_to_dir + 'calc_27_7_28')

os.chdir(path_to_dir + 'calc_new_28_7_28_narrow')


for file in glob.glob("*.csv"):
    print(file)    
files = [x for x in glob.glob("*.csv")]


In [ ]:
dfs = []

for file_name in files:
    this_list = []
    
    file_name_splitted = file_name.split('_')
    this_id = int(file_name_splitted[-3])
    this_head = int(file_name_splitted[-1].replace('.csv', ''))
    this_name = api.ESP_name(this_id)
    this_list += [this_id, this_head, this_name]
    
    one_file = pd.read_csv(file_name, index_col = [0])
    one_file['this_id'] = this_id
    one_file['this_head'] = this_head
    one_file['this_name'] = this_name
    dfs.append(one_file.copy())

    
big_df = pd.concat(dfs)

big_df['super_id'] = big_df['this_name'].astype(str) + ' H = ' + big_df['this_head'].astype(str) + ' м'


In [ ]:
big_df

In [ ]:
big_df.iloc[0]

In [ ]:
big_df['nodal_error_p_dis'] = abs(big_df['lp_esp_dis'] - big_df['lp_esp_dis_by_tube'])

In [ ]:
pumps_in_big_df = db_df[db_df['ID'].isin(big_df['this_id'].astype(int).unique())].copy()
pumps_in_big_df.loc[:, 'max_eff'] = pumps_in_big_df['eff_points'].apply(lambda x: max(x))
pumps_in_big_df

In [ ]:
pumps_in_big_df.T

In [ ]:
for_add = pumps_in_big_df.copy()

In [ ]:
for_add.T

In [ ]:
for_add = for_add.drop(columns = for_add.columns[12:])
for_add = for_add.drop(columns = ['source'])
for_add = for_add.reset_index(drop = True)
for_add

In [ ]:
for_add = for_add[for_add['ID'] != 684]
for_add

In [ ]:
for_add.columns = ['ID', 'Производитель', 'Название ЭЦН', 'Макс. число ступеней', 'Подача (ном), м3/сут', 'Левая граница, м3/сут',
                  'Правая граница, м3/сут', 'Макс. подача, м3/сут', 'Частота вращения, об/мин', 'Частота (ном), Гц', 'КПД (макс.), д.ед.']
for_add.T

In [ ]:
sweet_table = pumps_in_big_df[['manufacturer', 'name', 'rate_nom_sm3day', 'freq_Hz']].copy()
#sweet_table = sweet_table.reset_index(drop=True)
sweet_table.columns = ['Производитель', 'Имя ЭЦН', 'Номинальная подача, м3/сут', 'Частота, Гц']
sweet_table

In [ ]:
for i in ['pi_sm3dayatm', 'p_bhp_atm', 'pres_atma']:
    big_df[i] = big_df['lparams'].apply(lambda x: json.loads(x)[i])
    

In [ ]:
big_df = big_df[big_df['nodal_error_p_dis']<5]
big_df

In [ ]:
big_df['this_id'].unique()

### Сводные таблицы

In [ ]:
pt_count = big_df[big_df['nodal_error_p_dis']<5].pivot_table(index = ['this_name', 'this_head'], values = ['leff', 'lqliq'], aggfunc  = 'count')
pt_count

In [ ]:
def calc_emv(x):
    return sum(x / len(x))
def calc_std(x):
    return np.std(x)

In [ ]:
pt = big_df.pivot_table(index = ['this_name', 'this_head'], values = ['leff', 'lqliq'], aggfunc  = calc_emv)
pt

#### КПД

In [ ]:
#import matplotlib.style
#import matplotlib as mpl
#mpl.style.use('ggplot')

In [ ]:
font = {#'family' : 'normal',
        #'weight' : 'bold',
        'size'   : 25}

matplotlib.rc('font', **font)

In [ ]:
from matplotlib.legend_handler import HandlerBase

In [ ]:
fig = plt.Figure()

pt_eff = big_df.pivot_table(index = ['this_name'],columns=['this_head'], values = ['leff'], aggfunc  = calc_emv)

pt_eff_std = big_df.pivot_table(index = ['this_name'],columns=['this_head'], values = ['leff'], aggfunc  = calc_std)

pt_eff.plot(kind='bar', yerr = pt_eff_std)
plt.xlabel('ЭЦН')
plt.ylabel('КПД, %')
plt.title('Анализ режима работы по КПД ЭЦН')

cc = ['tab:blue', 'tab:orange', 'tab:green']
heads = [x[1] for x in pt_eff.columns]

colors = {f"{heads[x]} м": cc[x] for x in range(len(pt_eff.columns))}
labels = list(colors.keys())
handles = [plt.Rectangle((0,0),1,1, color=colors[label]) for label in labels]
plt.legend(handles, labels, title="Напор ЭЦН", loc=4, 
                     fontsize='small', fancybox=True)
plt.xticks(rotation = 45, ha ="right")


plt.grid()
pt_eff

#### Дебит жидкости

In [ ]:
pt_qliq = big_df.pivot_table(index = ['this_name'],columns=['this_head'], values = ['lqliq'], aggfunc  = calc_emv)
pt_qliq_std = big_df.pivot_table(index = ['this_name'],columns=['this_head'], values = ['lqliq'], aggfunc  = np.std)

hand = pt_qliq.plot(kind='bar', yerr = pt_qliq_std)
plt.xlabel('ЭЦН')
plt.ylabel('Дебит жидкости, м3/сут')
plt.title('Анализ режима работы скважин по дебиту жидкости')


cc = ['tab:blue', 'tab:orange', 'tab:green']
heads = [x[1] for x in pt_qliq.columns]

colors = {f"{heads[x]} м": cc[x] for x in range(len(pt_qliq.columns))}
labels = list(colors.keys())
handles = [plt.Rectangle((0,0),1,1, color=colors[label]) for label in labels]
plt.legend(handles, labels, title="Напор ЭЦН", loc=4, 
                     fontsize='small', fancybox=True)
plt.xticks(rotation = 45, ha ="right")


plt.grid()
pt_qliq

#### Деградация КПД - Наработка на отказ (ННО)

In [ ]:
os.chdir(r'C:\Git\probability_calculations')
with open('ESP_json.db', 'r') as outfile:
    file = outfile
    db = json.load(outfile)

db_df = pd.DataFrame(db)
db_df = db_df.T  

number_of_point_in_curves = []
for i in db_df.index:
    this_pump = db_df.loc[i]
    number_of_point_in_curves.append(len(this_pump['rate_points']))
    
db_df['number_of_point_in_curves'] = number_of_point_in_curves



this_pump = db_df[db_df['ID'] == 3256]
this_pump.T

In [ ]:
def find_max_eff_by_pump_id(ID: int):
    this_pump = db_df[db_df['ID'] == ID]
    f = interpolate.interp1d(this_pump['rate_points'].values[0], this_pump['eff_points'].values[0], kind = 'quadratic')

    q_liq_range = np.arange(min(this_pump['rate_points'].values[0]), max(this_pump['rate_points'].values[0]), 1)
    #q_liq_range = this_pump['rate_points'].values[0]
    eff_interpolated = f(q_liq_range)
    fig  = plt.Figure()

    plt.plot(this_pump['rate_points'].values[0], this_pump['eff_points'].values[0], 'o-', label = 'Исходные точки с БД')
    plt.plot(q_liq_range, eff_interpolated, label = 'Проинтерполированные значения')
    plt.legend()
    plt.xlabel('Подача, м3/сут')
    plt.ylabel('КПД, д.ед.')
    plt.title(f"Pump name = {this_pump['name'].values[0]} ID = {this_pump['ID'].values[0]}")
    plt.show()
    max_init = max(this_pump['eff_points'].values[0])
    max_interpolated = max(eff_interpolated)
    print(f"max init = {max_init} max interpolated = {max_interpolated}")
    print(f"eff diff = {abs(max_init - max_interpolated)}")
    return max_init, max_interpolated

find_max_eff_by_pump_id(3259)

In [ ]:
id_max_eff_mask = {}
for i in big_df['this_id'].unique():
    _, id_max_eff_mask[i]  = find_max_eff_by_pump_id(i)
id_max_eff_mask

In [ ]:
k_degr_eff_range = np.arange(0, 1, 0.05)
def nno(x):
    return 1 - x**2

#fig = plt.Figure()
plt.plot(k_degr_eff_range, nno(k_degr_eff_range))
plt.xlabel('$1 - K_{\eta}^{дегр}$ (Деградация по КПД)')
plt.ylabel('Коэффициент снижения наработки на отказ, д.ед.')
plt.title('Условная функция влияния снижения КПД на ННО ')
plt.grid()
plt.show()


In [ ]:
big_df['this_max_eff'] = big_df['this_id'].apply(lambda x: id_max_eff_mask[x])
big_df['k_degr_eff'] = (big_df['leff']/100) / big_df['this_max_eff']
big_df['eff_div_max_eff'] = (big_df['leff']/100) / big_df['this_max_eff']

big_df['k_degr_eff'] = nno(1 - big_df['k_degr_eff'])
big_df['k_degr_eff'].plot.hist(bins=100)
plt.xlabel('k_degr_eff')

In [ ]:
pt_k_degr_eff = big_df.pivot_table(index = ['this_name'],columns=['this_head'], values = ['k_degr_eff'], aggfunc  = calc_emv)
pt_k_degr_eff_std = big_df.pivot_table(index = ['this_name'],columns=['this_head'], values = ['k_degr_eff'], aggfunc  = np.std)


pt_k_degr_eff.plot(kind='bar', yerr=pt_k_degr_eff_std)
plt.legend(loc=3)
plt.xlabel('ЭЦН')
plt.ylabel('Kдегр КПД, д.ед.')



cc = ['tab:blue', 'tab:orange', 'tab:green']
heads = [x[1] for x in pt_qliq.columns]

colors = {f"{heads[x]} м": cc[x] for x in range(len(pt_qliq.columns))}
labels = list(colors.keys())
handles = [plt.Rectangle((0,0),1,1, color=colors[label]) for label in labels]
plt.legend(handles, labels, title="Напор ЭЦН", loc=4, 
                     fontsize='small', fancybox=True)
plt.xticks(rotation = 45, ha ="right")

plt.title('Анализ режима работы по деградации НРХ ЭЦН')
plt.grid()

pt_k_degr_eff

#### Финальная оценка

In [ ]:
pt_eff

In [ ]:
pt_eff_std

In [ ]:
def find_max_val_for_norm(pivot_table_df):
    pt_values_eff = pivot_table_df.values.flatten()
    pt_values_eff = pt_values_eff[~np.isnan(pt_values_eff)]
    pt_values_eff_max = pt_values_eff.max()
    return pt_values_eff_max

In [ ]:

pt_values_eff_max = find_max_val_for_norm(pt_eff)
pt_values_qliq_max = find_max_val_for_norm(pt_qliq)
pt_values_k_degr_eff_max = find_max_val_for_norm(pt_k_degr_eff)

pt_values_eff_std_max = find_max_val_for_norm(pt_eff_std)
pt_values_qliq_std_max = find_max_val_for_norm(pt_qliq_std)
pt_values_k_degr_eff_std_max = find_max_val_for_norm(pt_k_degr_eff_std)



print('pt_values_eff_max', pt_values_eff_max,
      'pt_values_qliq_max', pt_values_qliq_max,
      'pt_values_k_degr_eff_max', pt_values_k_degr_eff_max)

pt_result = pt_eff.copy()
pt_result = pt_eff.values/pt_values_eff_max  * pt_qliq.values/pt_values_qliq_max * pt_k_degr_eff.values/pt_values_k_degr_eff_max
pt_result_std = pt_eff_std.values/pt_values_eff_std_max  * pt_qliq_std.values / pt_values_qliq_std_max * pt_k_degr_eff_std.values / pt_values_k_degr_eff_std_max

pt_result = pd.DataFrame(pt_result, index = pt_eff.index, columns = [x[1] for x in pt_eff.columns])
pt_result_std = pd.DataFrame(pt_result_std, index = pt_eff.index, columns = [x[1] for x in pt_eff.columns])




pt_result.plot(kind='bar', yerr=pt_result_std, ylim=[0, 2])


cc = ['tab:blue', 'tab:orange', 'tab:green']
heads = [x[1] for x in pt_qliq.columns]

colors = {f"{heads[x]} м": cc[x] for x in range(len(pt_qliq.columns))}
labels = list(colors.keys())
handles = [plt.Rectangle((0,0),1,1, color=colors[label]) for label in labels]
plt.legend(handles, labels, title="Напор ЭЦН", loc=4, 
                     fontsize='small', fancybox=True)

plt.xticks(rotation = 45, ha ="right")


plt.ylabel('Финальный рейтинг')
plt.xlabel('ЭЦН')
plt.title('Сводный рейтинг ЭЦН')
plt.grid()
pt_result

In [ ]:
r = []
for i in pt_result.index:
    this_s = pt_result.loc[i]
    this_s.index = i + ' ' + this_s.index.astype(str)
    r.append(this_s)
pd.concat(r).sort_values()

In [ ]:
r = []
for i in pt_result.index:
    this_s = pt_result.loc[i]
    this_s.index = i + ' ' + this_s.index.astype(str)
    r.append(this_s)
pd.concat(r).sort_values(ascending=False).plot.bar()
plt.xlabel('ЭЦН')
plt.xticks(rotation = 45, ha ="right")

plt.ylabel('Финальный рейтинг')

In [ ]:
pt_result_normed = pt_result / pt_result_std
pt_result_normed

In [ ]:
r = []
for i in pt_result_normed.index:
    this_s = pt_result_normed.loc[i]
    this_s.index = i + ' H = ' + this_s.index.astype(str) + ' м'
    r.append(this_s)
pd.concat(r).sort_values(ascending=False).plot.bar()
plt.xlabel('ЭЦН')
plt.xticks(rotation = 45, ha ="right")

plt.ylabel('Финальный рейтинг')
plt.title('Финальный рейтинг подбора ЭЦН (комбинированный)')

## Распределения 

### КПД

In [ ]:
fig = plt.Figure()

for i in big_df['this_name'].unique():
    for j in big_df['this_head'].unique():
        this_df = big_df[(big_df['this_name'] == i)  &  (big_df['this_head'] == j)]

        plt.hist(this_df['leff'], bins = 100, density = True ,alpha=0.5, label = f"{i} H = {j} м")


plt.legend()
plt.title('Распределения для каждого тестируемого насоса')
plt.xlabel('КПД, д.ед.')
plt.show()

### Дебита жидкости

In [ ]:
fig = plt.Figure()

for i in big_df['this_name'].unique():
    for j in big_df['this_head'].unique():
        this_df = big_df[(big_df['this_name'] == i)  &  (big_df['this_head'] == j)].copy()

        plt.hist(this_df['lqliq'], bins = 100, density = True ,alpha=0.5, label = f"{i} H = {j} м")


plt.legend()
plt.title('Распределения для каждого тестируемого насоса')
plt.xlabel('Дебит жидкости, м3/сут')
plt.show()

### Доли газа на приеме

In [ ]:
fig = plt.Figure()

for i in big_df['this_name'].unique():
    for j in big_df['this_head'].unique():
        this_df = big_df[(big_df['this_name'] == i)  &  (big_df['this_head'] == j)]

        plt.hist(this_df['lgas_fraction_intake'], bins = 100, density = True ,alpha=0.5, label = f"{i} H = {j} м")


plt.legend()
plt.title('Доля газа на приеме эцн, %')
plt.xlabel('Доля газа на приеме эцн, %')
plt.show()

In [ ]:
fig = plt.Figure()

for i in big_df['this_name'].unique():
    for j in big_df['this_head'].unique():
        this_df = big_df[(big_df['this_name'] == i)  &  (big_df['this_head'] == j)]

        plt.hist(this_df['lgas_fraction_intake'], bins = 100, density = True ,alpha=0.5, label = f"{i} H = {j} м")


    plt.legend()
    plt.title('Доля газа на приеме эцн, %')
    plt.xlabel('Дебит жидкости, м3/сут')
    plt.show()

### Дебита ГЖС через насос

In [ ]:
fig = plt.Figure()

for i in big_df['this_name'].unique():
    for j in big_df['this_head'].unique():
        this_df = big_df[(big_df['this_name'] == i)  &  (big_df['this_head'] == j)]

        plt.hist(this_df['lq_mix_pump_mean'], bins = 100, density = True ,alpha=0.5, label = f"{i} H = {j} м")


plt.legend()
plt.title('Распределения для каждого тестируемого насоса')
plt.xlabel('Дебит ГЖС через насос, м3/сут')
plt.show()

### Распределения через KDE

In [ ]:
big_df

In [ ]:
fig, ax = plt.subplots()
big_df_t = big_df.copy()
big_df_t = big_df_t.rename(columns = {'super_id': 'ЭЦН'})
g = sns.kdeplot(data=big_df_t, x="lqliq", hue="ЭЦН", ax = ax)
plt.title('Распределения для каждого подбираемого насоса')
plt.xlabel('Дебит жидкости, м3/сут')
plt.ylabel('Плоность вероятности')
#ax.legend(title='2')


In [ ]:
fig, ax = plt.subplots()
big_df_t = big_df.copy()
big_df_t = big_df_t.rename(columns = {'super_id': 'ЭЦН'})
big_df_t['leff'] = big_df_t['leff'] /100
g = sns.kdeplot(data=big_df_t, x="leff", hue="ЭЦН", ax = ax)
plt.title('Распределения для каждого подбираемого насоса')
plt.xlabel('КПД, д.ед.')
plt.ylabel('Плоность вероятности')
#ax.legend(title='2')


In [ ]:
sns.kdeplot(data=big_df, x="pi_sm3dayatm", hue="super_id")
plt.title('PDF для каждого тестируемого насоса')
plt.xlabel('К прод., м3/сут/атм')

In [ ]:
sns.kdeplot(data=big_df, x="pres_atma", hue="super_id")
plt.title('PDF для каждого тестируемого насоса')
plt.xlabel('Пластовое давление, атм')

In [ ]:
sns.kdeplot(data=big_df, x="p_bhp_atm", hue="super_id")
plt.title('PDF для каждого тестируемого насоса')
plt.xlabel('Забойное давление, атм')


In [ ]:
sns.kdeplot(data=big_df, x="pres_atma", hue="super_id")
plt.title('PDF для каждого тестируемого насоса')
plt.xlabel('Пластовое давление, атм')

In [ ]:
sns.kdeplot(data=big_df, x="lgas_fraction_intake", hue="super_id")
plt.title('PDF для каждого тестируемого насоса')
plt.xlabel("Доля газа на приеме ЭЦН (1-ая ступень)")

In [ ]:
sns.kdeplot(data=big_df, x="leff", hue="super_id")
plt.title('PDF для каждого тестируемого насоса')
plt.xlabel('КПД, д.ед.')

In [ ]:
big_df

In [ ]:
sns.kdeplot(data=big_df, x="pi_sm3dayatm", hue="super_id")
plt.title('PDF для каждого тестируемого насоса')
plt.xlabel('К продуктивности, м3/сут/атм')

### КПД от дебита жидкости

In [ ]:
#font = {'size'   : 1}
#matplotlib.rc('font', **font)
plt.rcParams["font.size"] = 25

In [ ]:
#sns.set(font_scale=1.5) 
big_df_to_plot = big_df.copy()
big_df_to_plot = big_df_to_plot.rename(columns = {'lqliq': 'Дебит жидкости, м3/сут',
                                                 'leff': 'КПД,  д.ед.',
                                                 'super_id': 'ЭЦН',
                                                 })
#fig, ax = plt.subplots()

r = sns.jointplot(
    data=big_df_to_plot,
    x="Дебит жидкости, м3/сут", y= 'КПД,  д.ед.', hue="ЭЦН",
    kind="kde",
    legend = False,
    height=15,
      #ax = ax
)
#g.despine(left=True)
#0r.fig.legend(loc='upper left')
r.ax_joint.legend_._loc_used_default = False
r.ax_joint.legend_._loc_real =4
r.ax_joint.legend_._fontsize =4
#r.ax_joint.legend_._in_layout = False

#r.ax_joint.legend_._bbox_to_anchor=(0.85, -0.1)

In [ ]:
#sns.set(font_scale=1.5) 
big_df_to_plot = big_df.copy()
big_df_to_plot = big_df_to_plot.rename(columns = {'lqliq': 'Дебит жидкости, м3/сут',
                                                 'leff': 'ЭЦН,  д.ед.',
                                                 'super_id': 'ЭЦН',
                                                 })
sns.jointplot(
    data=big_df_to_plot,
    x="Дебит жидкости, м3/сут", y= 'ЭЦН,  д.ед.', hue="ЭЦН",
    #kind="kde",
    legend = True,
    height=15
)


In [ ]:
#sns.set(font_scale=1.5) 
big_df_to_plot = big_df[big_df['nodal_error_p_dis'] <2].copy()
big_df_to_plot = big_df_to_plot.rename(columns = {'lqliq': 'Дебит жидкости, м3/сут',
                                                 'leff': 'КПД,  д.ед.',
                                                 'super_id': 'ЭЦН',
                                                 })
sns.jointplot(
    data=big_df_to_plot,
    x="Дебит жидкости, м3/сут", y= 'КПД,  д.ед.', hue="ЭЦН",
    #kind="kde",
    legend = True,
    height=15
)

In [ ]:
#sns.set(font_scale=1.5) 
big_df_to_plot = big_df.copy()
big_df_to_plot = big_df_to_plot.rename(columns = {'lqliq': 'Дебит жидкости, м3/сут',
                                                 'leff': 'КПД,  д.ед.',
                                                 'super_id': 'ЭЦН',
                                                 })
sns.displot(
    data=big_df_to_plot,
    x="Дебит жидкости, м3/сут", y= 'КПД,  д.ед.', hue="ЭЦН",
    #kind="kde",
    binwidth=(0.1, 0.1),
    legend = True,
    height=15
)


In [ ]:
#sns.set(font_scale=1.5) 
big_df_to_plot = big_df.copy()
big_df_to_plot['this_id'] = big_df_to_plot['this_id'].astype(str)
big_df_to_plot = big_df_to_plot[big_df_to_plot['this_id'].isin(['685'])]
big_df_to_plot = big_df_to_plot.rename(columns = {'lqliq': 'Дебит жидкости, м3/сут',
                                                 'leff': 'КПД,  д.ед.',
                                                 'super_id': 'ЭЦН',
                                                 })
sns.displot(
    data=big_df_to_plot,
    x="Дебит жидкости, м3/сут", y= 'КПД,  д.ед.', hue="this_id",
    #kind="kde",
    binwidth=(0.1, 0.1),
    legend = True,
    height=15
)


In [ ]:
big_df['super_id'].unique()

In [ ]:
distplot.__dict__

In [ ]:
distplot._axes[0][0].xaxis.__dict__

In [ ]:
plt.rcParams["font.size"] = 20

In [ ]:
#sns.set(font_scale=1.5) 
big_df_to_plot = big_df[big_df['nodal_error_p_dis']<3].copy()
big_df_to_plot = big_df_to_plot[big_df_to_plot['super_id'].isin([
    
    '0215ЭЦНАКИ5А-125И H = 2600 м',
       '0215ЭЦНАКИ5А-125И H = 2800 м', '0215ЭЦНАКИ5А-125И H = 3000 м',
    
    '660 H = 2600 м', '660 H = 2800 м', '660 H = 3000 м',
'ЭЦН5-80 H = 2600 м', 'ЭЦН5-80 H = 2800 м', 'ЭЦН5-80 H = 3000 м'
])]
big_df_to_plot = big_df_to_plot[big_df_to_plot['this_id'] != 685]
big_df_to_plot = big_df_to_plot.rename(columns = {'lqliq': 'Дебит жидкости, м3/сут',
                                                 'leff': 'КПД, д.ед.',
                                                 'super_id': 'ЭЦН',
                                                 })
distplot = sns.displot(
    data=big_df_to_plot,
    x="Дебит жидкости, м3/сут", y= 'КПД, д.ед.', hue="ЭЦН",
    #kind="kde",
    binwidth=(0.1, 0.1),
    legend = True,
    height=15,
)

plt.xlim((55, 140))
plt.ylim((15, 43))

In [ ]:
#sns.set(font_scale=1.5) 
big_df_to_plot = big_df.copy()
big_df_to_plot = big_df_to_plot.rename(columns = {'lqliq': 'Дебит жидкости, м3/сут',
                                                 'leff': 'КПД, д.ед.',
                                                 'super_id': 'ЭЦН',
                                                 })
sns.jointplot(
    data=big_df_to_plot,
    x="Дебит жидкости, м3/сут", y= 'КПД, д.ед.', hue="ЭЦН",
    #kind="kde",
    #binwidth=(0.1, 0.1),
    legend = True,
    height=15
)

In [ ]:
big_df_to_plot.iloc[0]

In [ ]:
big_df_to_plot

In [ ]:
#sns.set(font_scale=1.5) 
big_df_to_plot = big_df.copy()
big_df_to_plot = big_df_to_plot.rename(columns = {'lqliq': 'Дебит жидкости, м3/сут',
                                                 'leff': 'ЭЦН,  д.ед.',
                                                 'super_id': 'ЭЦН',
                                                  'lq_mix_pump_mean': 'Средний дебит ГЖС через ЭЦН, м3/сут'
                                                 })


sns.jointplot(
    data=big_df_to_plot,
    x='Средний дебит ГЖС через ЭЦН, м3/сут', y= 'ЭЦН,  д.ед.', hue="ЭЦН",
    kind="kde",
    legend = True,
    height=15,
    xlim = (40, 170)
)



In [ ]:
fig, ax = plt.subplots()
big_df[big_df['nodal_error_p_dis']<0.5].plot.scatter(x="lqliq", y="leff", c="lpower_esp", legend=True, colormap='viridis', ax=ax)
plt.legend()
plt.xlabel('Дебит жидкости, м3/сут')
plt.ylabel('КПД, д.ед.')

plt.show()

### Боксплоты

In [ ]:
{x: big_df[big_df['super_id'] == x]['leff'].reset_index(drop=True) for x in big_df['super_id'].unique()}['ЭЦН5-80 H = 2600 м']

In [ ]:
df_for_box = pd.DataFrame({x: big_df[big_df['super_id'] == x]['leff'].reset_index(drop=True)  for x in big_df['super_id'].unique()},
                          
                          
                          index=range(10000))
df_for_box.plot.box()
plt.xticks(rotation = 45)
plt.ylabel('КПД, д.ед.')
plt.show()

In [ ]:
df_for_box = pd.DataFrame({x: big_df[big_df['super_id'] == x]['lqliq'].reset_index(drop=True)  for x in big_df['super_id'].unique()},
                          
                          index=range(3000))
df_for_box.plot.box()
plt.xticks(rotation = 45)
plt.ylabel('Дебит жидкости, д.ед.')
plt.show()

In [ ]:
big_df

### Violin plot

In [ ]:
sns.violinplot(x="super_id", y="leff", hue="this_head",
                    data=big_df,
               #palette="muted",
               #split=True
               #scale="lqliq"
              )
plt.xticks(rotation = 45)
plt.show()

In [ ]:
sns.violinplot(x="super_id", y="lqliq", hue="this_head",
                    data=big_df,
               #width = 0.8,
               #palette="muted",
               #split=True
               #scale="lqliq"
               #scale='width'
              )
plt.xticks(rotation = 45)
plt.show()

## Один насос

In [ ]:
pumps_in_big_df

In [ ]:
this_pump

In [ ]:
this_pump = db_df[db_df['ID'] == 1153].iloc[0]

q_arr = np.array(this_pump['rate_points'])
h_esp_arr = np.array(this_pump['head_points'])
power_esp_arr = np.array(this_pump['power_points']) 
efficiency_esp_arr = np.array(this_pump['eff_points'])

well_model.plot_pump_curve(q_arr,
    h_esp_arr,
    power_esp_arr,
    efficiency_esp_arr,
    1,
    this_pump['name'],
    )

In [ ]:
 big_df['super_id'].unique()

In [ ]:
this_super_id = 'ЭЦН5-80 H = 2800 м'

In [ ]:
one_pump = big_df[big_df['super_id'] == this_super_id]
one_pump.plot.scatter(x = 'lqliq', y = 'leff')

In [ ]:
big_df.plot.scatter(x="lqliq", y="leff", c="lgas_fraction_intake", legend=True, colormap='viridis')
plt.legend()
plt.xlabel('Дебит жидкости, м3/сут')
plt.ylabel('КПД, д.ед.')
plt.title('КПД ЭЦН от дебита жидкости')

plt.show()

In [ ]:
big_df.plot.scatter(x="lqliq", y="lgas_fraction_intake", c="leff", colormap='viridis')
plt.xlabel('Дебит жидкости, м3/сут')
plt.ylabel('Доля газа на приеме ЭЦН, д.ед.')
plt.legend()
plt.title('Доля газа на приеме от дебита жидкости')
plt.show()

In [ ]:
fig, ax = plt.subplots()

big_df.plot.scatter(ax = ax, x="lq_mix_pump_mean", y="lgas_fraction_intake", c="leff", colormap='viridis')



plt.xlabel('Дебит жидкости, м3/сут')

plt.ylabel('Доля газа на приеме ЭЦН, д.ед.')


plt.title('Дебит жидкости от доли газа на приеме от дебита жидкости')

plt.show()

In [ ]:
fig, ax = plt.subplots()

big_df.plot.scatter(ax = ax, x="lgas_fraction_intake", y="leff", 
                    c="lq_mix_pump_mean",
                    colormap='viridis')


plt.xlabel('Доля газа на приеме ЭЦН, д.ед.')

plt.ylabel('КПД, д.ед.')



plt.title('Дебит жидкости от доли газа на приеме от дебита жидкости')

plt.show()

In [ ]:
fig, ax = plt.subplots()

big_df.plot.scatter(ax = ax, x="lqliq", y="lp_bhp_atm", c="leff", colormap='viridis')
plt.xlabel('Дебит жидкости, м3/сут')
plt.ylabel('Забойное давление, атм')
plt.legend()
plt.title('Доля газа на приеме от дебита жидкости')
plt.show()
#pi_sm3dayatm

In [ ]:
fig, ax = plt.subplots()

big_df.plot.scatter(ax = ax, x="pi_sm3dayatm", y="lqliq", c="leff", colormap='viridis')
plt.xlabel('Коэффициент продуктивности, м3/сут/атм')
plt.ylabel('Дебит жидкости, м3/сут')
plt.legend()
plt.title('Дебит жидкости от коэффициента продуктивности')
plt.show()
#pi_sm3dayatm

In [ ]:
fig, ax = plt.subplots()

big_df.plot.scatter(ax = ax, x="pres_atma", y="lqliq", c="leff", colormap='viridis')
plt.xlabel('Пластовое давление, атм')
plt.ylabel('Дебит жидкости, м3/сут')
plt.legend()
plt.title('Дебит жидкости от пластового давления')
plt.show()

In [ ]:
fig, ax = plt.subplots()

big_df.plot.scatter(ax = ax, x="lgas_fraction_intake", y="leff", c="lqliq", colormap='viridis')

ax.legend(list(big_df['super_id'].unique()))


plt.ylabel('КПД, д.ед.')

plt.xlabel('Доля газа на приеме ЭЦН, д.ед.')


plt.title('КПД от доли газа на приеме от дебита жидкости')


plt.show()

In [ ]:
fig, ax = plt.subplots()

big_df.plot.scatter(ax = ax, x="lgas_fraction_intake", y="lqliq", c="leff", colormap='viridis')



plt.ylabel('Дебит жидкости, м3/сут')

plt.xlabel('Доля газа на приеме ЭЦН, д.ед.')


plt.title('Дебит жидкости от доли газа на приеме от дебита жидкости')

plt.show()

In [ ]:
fig,ax = plt.subplots()
sns.scatterplot(data=big_df[big_df['nodal_error_p_dis']<1].copy(), hue='super_id', x='lgas_fraction_intake', y='lqliq')
plt.legend(loc=2)
plt.savefig('scatter.png')
plt.show()

In [ ]:
fig,ax = plt.subplots()
sns.scatterplot(data=big_df, hue='super_id', x='lqliq', y='lgas_fraction_intake')
plt.legend(loc=2)
plt.savefig('scatter.png')
plt.show()

In [ ]:
fig,ax = plt.subplots()
sns.scatterplot(data=big_df, hue='leff', x='lgas_fraction_intake', y='lqliq')
plt.legend(loc=2)
plt.show()

In [ ]:
this_frac_from_big_df = big_df[abs(big_df['lp_esp_dis']-big_df['lp_esp_dis_by_tube']) >20]
fig,ax = plt.subplots()
sns.scatterplot(data=this_frac_from_big_df,
                            
                            hue='super_id', x='lp_esp_dis', y='lp_esp_dis_by_tube')
plt.legend(loc=2)
plt.show()
this_frac_from_big_df

In [ ]:
big_df.columns

In [ ]:
this_frac_from_big_df = big_df[abs(big_df['lp_esp_dis']-big_df['lp_esp_dis_by_tube']) > 0]
print(f"len = {this_frac_from_big_df.shape[0]}")
fig,ax = plt.subplots()
sns.scatterplot(data=this_frac_from_big_df,
                            
                            hue='super_id', x='lp_esp_dis_by_tube', y='lp_esp_dis')
plt.legend(loc=1)
plt.show()

In [ ]:
this_frac_from_big_df = big_df[abs(big_df['lp_esp_dis']-big_df['lp_esp_dis_by_tube']) > 0]
print(f"len = {this_frac_from_big_df.shape[0]}")
fig,ax = plt.subplots()
sns.scatterplot(data=this_frac_from_big_df,
                            
                            hue='super_id', x='lq_mix_pump_mean', y='lp_esp_dis')
plt.legend(loc=1)
plt.show()


In [ ]:
this_frac_from_big_df = big_df[abs(big_df['lp_esp_dis']-big_df['lp_esp_dis_by_tube']) > 0]
print(f"len = {this_frac_from_big_df.shape[0]}")
fig,ax = plt.subplots()
sns.scatterplot(data=this_frac_from_big_df,
                            
                            hue='super_id', x='pi_sm3dayatm', y='lgas_fraction_intake')
plt.legend(loc=1)
plt.show()

In [ ]:
this_frac_from_big_df = big_df[abs(big_df['lp_esp_dis']-big_df['lp_esp_dis_by_tube']) > 0]
print(f"len = {this_frac_from_big_df.shape[0]}")
fig,ax = plt.subplots()
sns.scatterplot(data=this_frac_from_big_df,
                            
                            hue='super_id', x='pi_sm3dayatm', y='this_head')
plt.legend(loc=1)
plt.show()

In [ ]:
this_frac_from_big_df = big_df[abs(big_df['lp_esp_dis']-big_df['lp_esp_dis_by_tube']) > 0]
print(f"len = {this_frac_from_big_df.shape[0]}")
fig,ax = plt.subplots()
sns.scatterplot(data=this_frac_from_big_df,
                            
                            hue='super_id', x='pi_sm3dayatm', y='lp_bhp_atm')
plt.legend(loc=1)
plt.show()

In [ ]:
this_frac_from_big_df = big_df[abs(big_df['lp_esp_dis']-big_df['lp_esp_dis_by_tube']) > 0]
print(f"len = {this_frac_from_big_df.shape[0]}")
fig,ax = plt.subplots()
sns.scatterplot(data=this_frac_from_big_df,
                            
                            hue='super_id', x='lp_bhp_atm', y='lgas_fraction_intake')
plt.legend(loc=1)
plt.show()

In [ ]:
this_frac_from_big_df = big_df[abs(big_df['lp_esp_dis']-big_df['lp_esp_dis_by_tube']) > 0]
print(f"len = {this_frac_from_big_df.shape[0]}")
fig,ax = plt.subplots()
sns.scatterplot(data=this_frac_from_big_df,
                            
                            hue='super_id', x='pres_atma', y='lgas_fraction_intake')
plt.legend(loc=1)
plt.show()

In [ ]:
this_frac_from_big_df = big_df[abs(big_df['lp_esp_dis']-big_df['lp_esp_dis_by_tube']) > 0]
print(f"len = {this_frac_from_big_df.shape[0]}")
fig,ax = plt.subplots()
sns.scatterplot(data=this_frac_from_big_df,
                            
                            hue='super_id', x='pres_atma', y='leff')
plt.legend(loc=3)
plt.show()

In [ ]:
this_frac_from_big_df = big_df[abs(big_df['lp_esp_dis']-big_df['lp_esp_dis_by_tube']) > 0]
print(f"len = {this_frac_from_big_df.shape[0]}")
fig,ax = plt.subplots()
sns.scatterplot(data=this_frac_from_big_df,
                            
                            hue='super_id', x='pi_sm3dayatm', y='leff')
plt.legend(loc=3)
plt.show()

In [ ]:
this_frac_from_big_df['super_id'].unique()

In [ ]:
this_super_id = this_frac_from_big_df['super_id'].unique()

In [ ]:
this_frac_from_big_df = big_df[abs(big_df['lp_esp_dis']-big_df['lp_esp_dis_by_tube']) > 0]
this_frac_from_big_df  = this_frac_from_big_df[this_frac_from_big_df['super_id'] ==this_frac_from_big_df['super_id'].unique()[0] ]
print(f"len = {this_frac_from_big_df.shape[0]}")
fig,ax = plt.subplots()
sns.scatterplot(data=this_frac_from_big_df,
                            
                            hue='super_id', x='pi_sm3dayatm', y='leff')
plt.legend(loc=3)
plt.show()

In [ ]:
this_frac_from_big_df = big_df[abs(big_df['lp_esp_dis']-big_df['lp_esp_dis_by_tube']) > 0]
this_frac_from_big_df['error'] = abs(big_df['lp_esp_dis']-big_df['lp_esp_dis_by_tube'])
print(f"len = {this_frac_from_big_df.shape[0]}")
fig,ax = plt.subplots()
sns.scatterplot(data=this_frac_from_big_df,
                            
                            hue='super_id', x='lq_mix_pump_mean', y='leff')
plt.legend(loc=3)
plt.show()

In [ ]:
this_frac_from_big_df = big_df[abs(big_df['lp_esp_dis']-big_df['lp_esp_dis_by_tube']) > 0]
this_frac_from_big_df['error'] = abs(big_df['lp_esp_dis']-big_df['lp_esp_dis_by_tube'])
print(f"len = {this_frac_from_big_df.shape[0]}")
fig,ax = plt.subplots()
sns.scatterplot(data=this_frac_from_big_df,
                            
                            hue='super_id', x='lqliq', y='leff')
plt.legend(loc=3)
plt.show()

In [ ]:
this_frac_from_big_df = big_df[abs(big_df['lp_esp_dis']-big_df['lp_esp_dis_by_tube']) > 0]
this_frac_from_big_df['error'] = abs(big_df['lp_esp_dis']-big_df['lp_esp_dis_by_tube'])
print(f"len = {this_frac_from_big_df.shape[0]}")
fig,ax = plt.subplots()
sns.scatterplot(data=this_frac_from_big_df,
                            
                            hue='super_id', x='lq_mix_pump_mean', y='error')
plt.legend(loc=1)
plt.show()

In [ ]:
this_frac_from_big_df.columns

In [ ]:
list(big_df['super_id'].unique())

In [ ]:
#sns.set_theme(style="ticks")
sns.pairplot(big_df[['leff', 'lqliq', 'lgas_fraction_intake', 'super_id']].copy(), hue="super_id", height=15)

In [ ]:
fig,ax = plt.subplots()
sns.scatterplot(data=this_frac_from_big_df,
                            
                            hue='super_id', x='leff', y='this_max_eff')
plt.legend(loc=3)
plt.show()

In [ ]:
fig,ax = plt.subplots()
sns.scatterplot(data=this_frac_from_big_df,
                            
                            hue='super_id', x='leff', y='k_degr_eff')
plt.legend(loc=3)
plt.show()

In [ ]:
fig,ax = plt.subplots()
sns.scatterplot(data=this_frac_from_big_df,
                            
                            hue='super_id', x='leff', y='eff_div_max_eff')
plt.legend(loc=3)
plt.show()

In [ ]:
fig,ax = plt.subplots()
sns.scatterplot(data=this_frac_from_big_df,
                            
                            hue='super_id', x='lgas_fraction_intake', y='eff_div_max_eff')
plt.legend(loc=3)
plt.show()

# Проверка расчета

In [ ]:
big_df['super_id'].unique()

In [ ]:
pump_super_id = '226ЭЦНАКИ5-125 H = 1600 м'
pump_super_id =  '10.1ЭЦНД5А-100 H = 1600 м'
pump_super_id = 'R-7 H = 2600 м'
pump_super_id = '0215ЭЦНАКИ5А-125И H = 3000 м'
pump_super_id = 'ЭЦН5-80 H = 2800 м'

one_pump_df_for_check_all = big_df[big_df['super_id'] == pump_super_id].copy()
#one_pump_df_for_check_all = big_df.copy()

one_pump_df_for_check = one_pump_df_for_check_all
#one_pump_df_for_check   = one_pump_df_for_check_all[one_pump_df_for_check_all['leff'] == one_pump_df_for_check_all['leff'].max()]
#one_pump_df_for_check   = one_pump_df_for_check_all[abs(one_pump_df_for_check_all['lp_esp_dis'] - one_pump_df_for_check_all['lp_esp_dis_by_tube']) >10]

one_pump_df_for_check

In [ ]:
one_pump_df_for_check

In [ ]:
one_case = one_pump_df_for_check.iloc[0]
one_case

In [ ]:
json.loads(one_case['lesp_df'])

In [ ]:
params = json.loads(one_case['lparams'])
params['n_dots_for_nodal'] = 20
params

In [ ]:
for i in params.keys():
    print(i, 'params', params[i], 'params_saved', params_saved[i])

In [ ]:
2800 / api_new.ESP_head_m(80, pump_id=1185)

In [ ]:
casing_pipe, tube_pipe, h_mes, head_esp, eff, power_esp, q_liq, status, gas_fraction_intake, p_bhp_atm, \
                            p_dis, p_esp_dis, json_params, q_mix_pump_mean, esp_df = well_model.calc_all(params, debug=1,
                                                                                              vba_version='7.28',
                                                                                             api=api, api_new=api_new)

In [ ]:
p_dis, p_esp_dis

In [ ]:
esp_df

In [ ]:
this_pump = db_df[db_df['ID'] == params['pump_id']].iloc[0]

q_arr = np.array(this_pump['rate_points'])
h_esp_arr = np.array(this_pump['head_points'])
power_esp_arr = np.array(this_pump['power_points']) 
efficiency_esp_arr = np.array(this_pump['eff_points'])
h_esp_arr_div_power_esp_arr = h_esp_arr/power_esp_arr
well_model.plot_pump_curve(q_arr,
    h_esp_arr,
    power_esp_arr,
    efficiency_esp_arr,
    z=1,
    esp_name= api.ESP_name(params['pump_id']),
    f=53,
    fnom=50,
    q_work=None,
    show=True,
    xlabel=None)

In [ ]:
fig, ax = plt.subplots()

ax.plot(q_arr * 53/50,efficiency_esp_arr*100, 'ro-', label ='БД НРХ' )
one_pump_df_for_check_all.plot.scatter(x = 'lq_mix_pump_mean', y = 'leff', ax =ax
                                      )

ax.legend()


In [ ]:
one_pump_df_for_check_all['lgas_fraction_intake'].plot.hist(bins=100)
plt.xlabel('Доля газа на приеме')

In [ ]:
one_pump_df_for_check_all['eff_div_max_eff'].plot.hist(bins=100)
plt.xlabel('eff_div_max_eff')

In [ ]:
one_pump_df_for_check_all

In [ ]:
api.ESP_name(params['pump_id'])

In [ ]:
api_new.ESP_name(params['pump_id'])

In [ ]:
one_case['this_name']

In [ ]:
eff

In [ ]:
one_case['leff']

In [ ]:
one_case

## Сборка результирующего DataFrame (deprecated)

In [ ]:
params

In [ ]:

df = well_model.save_in_df(lh_mes, lhead_esp, leff, lpower_esp, lqliq, lstatus, lgas_fraction_intake, 
                   lp_bhp_atm, lp_esp_dis, lparams, num_simulations, lp_esp_dis_by_tube)
df

In [ ]:
debug_params = json.loads(df['lparams'][0])
debug_params['n_dots_for_nodal'] = 15
debug_params['esp_head_m'] = 1000
debug_params['num_stages'] = calc_num_stages(debug_params)
r = calc_all(debug_params)




In [ ]:
#построение графиков
#for i in df.columns:
#    #df = df.sort_values(by = i)
#    if type(df.reset_index()[i].values[0]) is not str:
#        df.reset_index()[i].plot(label = i)
#        plt.ylabel(i)
#        plt.xlabel('Порядковый номер')
#        plt.legend()
#        plt.show()

## Построение итоговых графиков  (deprecated)

In [ ]:
print(f"Error = {len(df['lstatus'][df['lstatus'] == 0])} из {df.shape[0]} или из {num_simulations}")
print('h_pump_m', params['h_pump_m'])
print('pump_id', params['pump_id'])
print('ESP_optRate_m3day', api.ESP_optRate_m3day(pump_id=params['pump_id']))
print('ESP_name', api.ESP_name(pump_id=params['pump_id']))



for i in [[lh_mes, 'Глубина спуска, м'],
          [lhead_esp, 'Напор, м'], 
          [leff, 'КПД, д.ед.'], 
          [lpower_esp, 'Мощность, кВт'],
          [lqliq, 'Дебит жидкости, м3/сут'],
          [lgas_fraction_intake, 'Доля газа на приеме, д.ед.'],
          [lp_bhp_atm, 'Забойное давление, атм'],
          [lp_esp_dis, 'Давление на выкиде, атм']
         ]:

    fig = plt.Figure()

    fig, ax = plt.subplots()
    
    label = i[1]
    x = np.array(i[0],dtype= np.float64)
    count, bins, ignored = plt.hist(x[~np.isnan(x)], bins=50,  density=True, label=label)
    
    
    # Расчет EMV
    leff2 = np.array(i[0], dtype = np.float64)
    leff2 = leff2[~np.isnan(leff2)]
    prob = leff2 * 0 + 1/len(leff2)

    emv = np.sum(leff2 * prob)


    plt.title(f"Распределение {label}, EMV = {round(emv, 3)}")
    plt.xlabel(label)
    ax.legend()
    plt.show()

In [ ]:
pump_id = params['pump_id']
#pump_id = 1460 #125
#pump_id = 1461 #200
#pump_id = 1868 #80
#pump_id = 2089 #160
#pump_id = 2289 #60
#pump_id = 2753 #100

qliq = np.arange(0, api.ESP_optRate_m3day(pump_id=pump_id)*2, 5)
eff = np.array([api.ESP_eff_fr(x, pump_id = pump_id, mu_cSt=1) for x in qliq])
power = np.array([api.ESP_power_W(x, pump_id = pump_id) for x in qliq])
h = np.array([api.ESP_head_m(x, pump_id = pump_id) for x in qliq])

qliq = qliq[eff >0]
power = power[eff >0]
h = h[eff >0]
eff = eff[eff >0]

plot_pump_curve(qliq, h, power, 
                eff, 1,
                api.ESP_name(pump_id=pump_id) + f" OPT Rate = {round(api.ESP_optRate_m3day(pump_id=pump_id), 2)}", f=50)

In [ ]:
plt.plot(qliq, h)

In [ ]:
plt.plot(eff/qliq, h/power)

In [ ]:
plt.plot(qliq, h/power)

In [ ]:
plt.plot(qliq,eff/qliq)

In [ ]:
plt.plot(qliq,eff, label = 'По характеристике')
plt.plot(qliq,h*1000*9.81*(qliq/86400)/power, label = 'По расчету через rho*g*h*Q/N')
plt.legend()
plt.title('Сравнение КПД прямой и косвенный расчет')
plt.ylabel('КПД, д.ед.')
plt.xlabel('Подача, м3/сут.')
plt.show()

# Анализ неоднозначности в НРХ

In [ ]:
os.chdir(r'C:\Git\probability_calculations')
with open('ESP_json.db', 'r') as outfile:
    file = outfile
    db = json.load(outfile)

In [ ]:
db_df = pd.DataFrame(db)
db_df = db_df.T
db_df

In [ ]:
for i in db_df.columns:
    print(i)

In [ ]:
db_df.iloc[738]

In [ ]:
db_df.columns

## Анализ распределения КПД

In [ ]:
db_df[db_df['eff_max']>0]['eff_max'].plot(kind='hist', bins = 100)

## Анализ количества точек для каждой НРХ

In [ ]:
number_of_point_in_curves = []
for i in db_df.index:
    this_pump = db_df.loc[i]
    number_of_point_in_curves.append(len(this_pump['rate_points']))
    
db_df['number_of_point_in_curves'] = number_of_point_in_curves

In [ ]:
db_df['number_of_point_in_curves'].plot(kind='hist', bins = 100)

In [ ]:
db_df_filtered

In [ ]:
font = {#'family' : 'normal',
        #'weight' : 'bold',
        'size'   : 15}
matplotlib.rc('font', **font)

In [ ]:
val = 50

db_df_filtered = db_df[db_df['number_of_point_in_curves'] <=val]
#db_df_filtered = db_df_filtered[db_df_filtered['number_of_point_in_curves'] >val-1]

#this_pump = db_df_filtered.iloc[2]
this_pump = db_df_filtered[db_df_filtered['ID'] == 1111].iloc[0]

plot_pump_curve(q_arr = np.array(this_pump['rate_points']), 
    h_esp_arr = np.array(this_pump['head_points']),
    power_esp_arr = np.array(this_pump['power_points']), 
    efficiency_esp_arr = np.array(this_pump['eff_points']),
         z = 1,
                esp_name = this_pump['name'], f=50)

## Анализ неоднозначности НРХ

### Построение НРХ с нормированной кривой

In [ ]:
def find_amount_of_change_direction(h_esp_arr):
    if type(h_esp_arr) is list:
        h_esp_arr = np.array(h_esp_arr)
    elif type(h_esp_arr) is str:
        h_esp_arr = np.array(json.loads(h_esp_arr))

    diff = np.diff(h_esp_arr)
    change_direction = diff[1:]*diff[:-1]
    
    amount_of_change_direction = len(change_direction[change_direction<0])
    return amount_of_change_direction


clear_db_df = db_df[db_df['number_of_point_in_curves']>=10]

this_pump = clear_db_df.iloc[700]
this_pump = clear_db_df[clear_db_df['ID'] == 3256].iloc[0]

q_arr = np.array(this_pump['rate_points'])
h_esp_arr = np.array(this_pump['head_points'])
power_esp_arr = np.array(this_pump['power_points']) 
efficiency_esp_arr = np.array(this_pump['eff_points'])
h_esp_arr_div_power_esp_arr = h_esp_arr/power_esp_arr



#plot_pump_curve(q_arr = q_arr, 
#    h_esp_arr = h_esp_arr,
#    power_esp_arr = power_esp_arr, 
#    efficiency_esp_arr = efficiency_esp_arr,
#         z = 1,
#    esp_name = this_pump['name'], f=50, show=True)




f=50
fnom=50
q_work=None
esp_name = this_pump['name']
z = 1

########## график
q_arr = q_arr * f/fnom
h_esp_arr = h_esp_arr * (f/fnom)**2
power_esp_arr = power_esp_arr * (f/fnom)**2
fig, ax = plt.subplots()
fig.subplots_adjust(right=0.75)

twin1 = ax.twinx()
twin2 = ax.twinx()
twin3 = ax.twinx()



# Offset the right spine of twin2.  The ticks and label have already been
# placed on the right by twinx above.
twin2.spines['right'].set_position(("axes", 1.1))
twin3.spines['right'].set_position(("axes", 1.2))

p1, = ax.plot(q_arr, h_esp_arr, "b-",  marker = 'o', label="Напор, м")
p2, = twin1.plot(q_arr, power_esp_arr, "r-",  marker = 'o',  label="Мощность, Вт")
p3, = twin2.plot(q_arr, efficiency_esp_arr, "g-",  marker = 'o', label="КПД, д.ед.")
p4, = twin3.plot(q_arr, h_esp_arr_div_power_esp_arr, "y-",  marker = 'o', label="Напор / Мощность")


if q_work is not None:
    f_interrr = interpolate.interp1d(q_arr,h_esp_arr, kind='cubic')
    p4 = ax.axvline(x=q_work, label=f"Рабочий режим Q={round(q_work, 2)}", linewidth=5, markersize=15)
    #p4, = ax.plot([q_work], [f_interrr(q_work)], "k",  marker = 'o', label="Рабочая точка",  markersize=15)

#ax.axvspan(esp_df['Левая граница'].values[0]*f/fnom, esp_df['Правая граница'].values[0]*f/fnom, 
#           alpha=0.2, color='green') TODO вытащить из БД

ax.set_xlabel("Подача, м3/сут")
ax.set_ylabel("Напор, м")
twin1.set_ylabel("Мощность, Вт")
twin2.set_ylabel("КПД, д.ед.")
twin3.set_ylabel("Напор / Мощность")

ax.yaxis.label.set_color(p1.get_color())
twin1.yaxis.label.set_color(p2.get_color())
twin2.yaxis.label.set_color(p3.get_color())
twin3.yaxis.label.set_color(p4.get_color())


tkw = dict(size=4, width=1.5)
ax.tick_params(axis='y', colors=p1.get_color(), **tkw)
twin1.tick_params(axis='y', colors=p2.get_color(), **tkw)
twin2.tick_params(axis='y', colors=p3.get_color(), **tkw)
twin3.tick_params(axis='y', colors=p4.get_color(), **tkw)

ax.tick_params(axis='x', **tkw)

if q_work is not None:
    ax.legend(handles=[p1, p2, p3, p4], loc='lower center')
else:
    ax.legend(handles=[p1, p2, p3, p4], loc='lower center')


ax.grid()

ax.set_title(f"{esp_name}, ступеней = {z} шт. при частоте = {f} Гц")
plt.show()



find_amount_of_change_direction(h_esp_arr), find_amount_of_change_direction(power_esp_arr), find_amount_of_change_direction(efficiency_esp_arr)

### Расчет количества перегибов на кривых и нормированной кривой для каждого ЭЦН

In [ ]:
clear_db_df['h_esp_arr_div_power_esp_arr'] = None
clear_db_df['power_points_left'] = None

for i in clear_db_df.index:
    
    clear_db_df.loc[i, 'h_esp_arr_div_power_esp_arr'] = json.dumps(list( np.array(clear_db_df.loc[i, 'head_points']) /  \
                                                        np.array(clear_db_df.loc[i, 'power_points'])))
    
    
    this_pump = clear_db_df.loc[i]

    power_points = np.array(this_pump['power_points'])
    rate_points = np.array(this_pump['rate_points'])

    power_points_left = power_points[rate_points< 0.3*max(rate_points)]
    
    
    clear_db_df.loc[i, 'power_points_left'] = json.dumps(list(power_points_left))

    
    

clear_db_df.loc[:, 'amount_of_change_direction_head'] = clear_db_df['head_points'].copy().apply(find_amount_of_change_direction)
clear_db_df.loc[:, 'amount_of_change_direction_eff'] = clear_db_df['eff_points'].copy().apply(find_amount_of_change_direction)
clear_db_df.loc[:, 'amount_of_change_direction_power'] = clear_db_df['power_points'].copy().apply(find_amount_of_change_direction)
clear_db_df.loc[:, 'amount_of_change_direction_h_esp_div_power'] = clear_db_df['h_esp_arr_div_power_esp_arr'].copy().apply(find_amount_of_change_direction)
clear_db_df.loc[:, 'amount_of_change_direction_power_points_left'] = clear_db_df['power_points_left'].copy().apply(find_amount_of_change_direction)
clear_db_df.loc[:, 'amount_of_change_direction_combined_solution'] = clear_db_df.apply(lambda x: min(x['amount_of_change_direction_power_points_left'], 
                                                                x['amount_of_change_direction_h_esp_div_power']), axis=1)


In [ ]:
clear_db_df

### Построение распределений для всей БД - количество перегибов для каждой кривой

In [ ]:
clear_db_df_with_calc = clear_db_df[clear_db_df['amount_of_change_direction_eff'] == 1]

In [ ]:
fig = plt.Figure()
ax = fig.add_axes([0,0,1,1])
for j, i in enumerate(clear_db_df_with_calc.columns[-6:]):
    v = clear_db_df_with_calc[i].value_counts() / clear_db_df_with_calc.shape[0]*100
    #v = clear_db_df_with_calc[i].value_counts() 
    v = v[v.index<5]
    plt.bar(v.index.values+0.1*j, v.values, label=i, width= 0.1, align='edge')
    plt.legend()
    #plt.show()
plt.grid()
plt.ylabel('Количество насосов')
plt.xlabel('Количество точек перегиба')
plt.show()

In [ ]:
#fig, ax = plt.subplots()

fig = plt.Figure()

#ax = fig.add_axes([0,0, 1, 1])
#for i in clear_db_df.columns[-4:]:
i = clear_db_df.columns[-1]
v = clear_db_df[i].value_counts()
plt.bar(v.index.values+0.00, v.values, label=i, width=0.25, align='center')

i = clear_db_df.columns[-2]
v = clear_db_df[i].value_counts()
plt.bar(v.index.values+0.25, v.values, label=i, width=0.25,  align='center')

i = clear_db_df.columns[-4]
v = clear_db_df[i].value_counts()
plt.bar(v.index.values+0.5, v.values, label=i, width=0.25,  align='center')
    
# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Scores')
ax.set_title('Scores by group and gender')

plt.legend()
#fig.tight_layout()

plt.show()

In [ ]:
clear_db_df[clear_db_df.columns[-1]].plot(kind='hist', bins = [0, 1, 2, 3, 4, 5], align='left', legend = 'Неопределенность по H/N')
clear_db_df[clear_db_df.columns[-2]].plot(kind='hist', bins = [0, 1, 2, 3, 4, 5], align='left', legend = 'Неопределенность по H/N')
plt.show()

### Построение конкретных кривых

In [ ]:
#clear_db_df_with_calc_filtered = clear_db_df[clear_db_df['amount_of_change_direction_eff']==1]
#clear_db_df_with_calc_filtered = clear_db_df[clear_db_df['amount_of_change_direction_power']==2]
#clear_db_df_with_calc_filtered = clear_db_df[clear_db_df['amount_of_change_direction_head']==1]
#clear_db_df_with_calc_filtered = clear_db_df[clear_db_df['amount_of_change_direction_head']==1]

clear_db_df_with_calc_filtered = clear_db_df[clear_db_df['amount_of_change_direction_h_esp_div_power']==1]
clear_db_df_with_calc_filtered = clear_db_df_with_calc_filtered.head(10)

print(clear_db_df_with_calc_filtered.shape)
for i in range(clear_db_df_with_calc_filtered.shape[0]):
    this_pump = clear_db_df_with_calc_filtered.iloc[i]


    q_arr = np.array(this_pump['rate_points'])
    h_esp_arr = np.array(this_pump['head_points'])
    power_esp_arr = np.array(this_pump['power_points']) 
    efficiency_esp_arr = np.array(this_pump['eff_points'])
    h_esp_arr_div_power_esp_arr = h_esp_arr/power_esp_arr



    #plot_pump_curve(q_arr = q_arr, 
    #    h_esp_arr = h_esp_arr,
    #    power_esp_arr = power_esp_arr, 
    #    efficiency_esp_arr = efficiency_esp_arr,
    #         z = 1,
    #    esp_name = this_pump['name'], f=50, show=True)

    f=50
    fnom=50
    q_work=None
    esp_name = this_pump['name']
    z = 1

    ########## график
    q_arr = q_arr * f/fnom
    h_esp_arr = h_esp_arr * (f/fnom)**2
    power_esp_arr = power_esp_arr * (f/fnom)**2
    fig, ax = plt.subplots()
    fig.subplots_adjust(right=0.75)

    twin1 = ax.twinx()
    twin2 = ax.twinx()
    twin3 = ax.twinx()



    # Offset the right spine of twin2.  The ticks and label have already been
    # placed on the right by twinx above.
    twin2.spines['right'].set_position(("axes", 1.1))
    twin3.spines['right'].set_position(("axes", 1.2))

    p1, = ax.plot(q_arr, h_esp_arr, "b-",  marker = 'o', label="Напор, м")
    p2, = twin1.plot(q_arr, power_esp_arr, "r-",  marker = 'o',  label="Мощность, Вт")
    p3, = twin2.plot(q_arr, efficiency_esp_arr, "g-",  marker = 'o', label="КПД, д.ед.")
    p4, = twin3.plot(q_arr, h_esp_arr_div_power_esp_arr, "y-",  marker = 'o', label="Напор / Мощность")


    if q_work is not None:
        f_interrr = interpolate.interp1d(q_arr,h_esp_arr, kind='cubic')
        p4 = ax.axvline(x=q_work, label=f"Рабочий режим Q={round(q_work, 2)}", linewidth=5, markersize=15)
        #p4, = ax.plot([q_work], [f_interrr(q_work)], "k",  marker = 'o', label="Рабочая точка",  markersize=15)

    #ax.axvspan(esp_df['Левая граница'].values[0]*f/fnom, esp_df['Правая граница'].values[0]*f/fnom, 
    #           alpha=0.2, color='green') TODO вытащить из БД

    ax.set_xlabel("Подача, м3/сут")
    ax.set_ylabel("Напор, м")
    twin1.set_ylabel("Мощность, Вт")
    twin2.set_ylabel("КПД, д.ед.")
    twin3.set_ylabel("Напор / Мощность")

    ax.yaxis.label.set_color(p1.get_color())
    twin1.yaxis.label.set_color(p2.get_color())
    twin2.yaxis.label.set_color(p3.get_color())
    twin3.yaxis.label.set_color(p4.get_color())


    tkw = dict(size=4, width=1.5)
    ax.tick_params(axis='y', colors=p1.get_color(), **tkw)
    twin1.tick_params(axis='y', colors=p2.get_color(), **tkw)
    twin2.tick_params(axis='y', colors=p3.get_color(), **tkw)
    twin3.tick_params(axis='y', colors=p4.get_color(), **tkw)

    ax.tick_params(axis='x', **tkw)

    if q_work is not None:
        ax.legend(handles=[p1, p2, p3, p4], loc='lower center')
    else:
        ax.legend(handles=[p1, p2, p3, p4], loc='lower center')


    ax.grid()

    ax.set_title(f"{esp_name}, ступеней = {z} шт. при частоте = {this_pump['freq_Hz']} Гц")
    plt.show()



    print(
        'h', find_amount_of_change_direction(h_esp_arr),
        'power', find_amount_of_change_direction(power_esp_arr), \
    'eff', find_amount_of_change_direction(efficiency_esp_arr), 
        'h/power', find_amount_of_change_direction(h_esp_arr_div_power_esp_arr),
        'final_solution', this_pump['amount_of_change_direction_combined_solution']
        
        
    )

In [ ]:
#график без номированной НРХ
plot_pump_curve(q_arr, h_esp_arr, power_esp_arr, efficiency_esp_arr, 1000, esp_name, f=50, fnom=50, q_work = None,
                   show = True)

### Расчет итоговой выборки

In [ ]:
clear_db_df_with_calc.shape[0], db_df.shape[0], clear_db_df.shape[0]
